# Import

In [1]:
from collections.abc import Iterator
from datetime import date, timedelta
from web3 import Web3
 
import boto3
import botocore
import logging
import numpy as np
import os
import pandas as pd
import string

# Download and Load Data

In [2]:
def iterate_dates(start_date: date, end_date: date) -> Iterator:
    '''Yields date in range of [start_date, end_date]

    Args:
        start_date: inclusive start date
        end_date: inclusive end date
    '''
    for n in range(int((end_date - start_date).days)+1):
        yield start_date + timedelta(n)

        
def download_nft_transfers(start_date: date, end_date: date, local_path: string) -> None:
    '''Downloads nft token transfer data from start_date to end_date

    Args:
        start_date: inclusive start date
        end_date: inclusive end date
        local_path: download path

    Raises:
        FileNotFoundError: if local_path does not exist
    '''
    
    if not os.path.exists(local_path):
        raise FileNotFoundError(f"The {local_path} does not exist")

    s3_client = boto3.client('s3')
    bucket_name = "nimble-data-warehouse-dev"
    for download_date in iterate_dates(start_date, end_date):
        str_date = download_date.strftime("%Y-%m-%d")
        remote_path = f"etl/ethereum/nft_token_transfers/date={str_date}/nft_token_transfers.csv"
        fname = os.path.join(local_path, f"nft_token_transfers={str_date}.csv")
        if not os.path.exists(fname):
            try:
                s3_client.download_file(bucket_name, remote_path, fname)
                logging.info(f"{remote_path} downloaded")
            except botocore.exceptions.ClientError as error:
                logging.error(f'{remote_path} not found')
            except botocore.exceptions.ParamValidationError as error:
                logging.error('The parameters you provided are incorrect: {}'.format(error))

                
def convert_value_from_wei(transfers: pd.DataFrame, unit: string) -> pd.DataFrame:
    '''Convert value in WEI to unit for transfers
        where possible units are 'kwei', 'mwei', 'gwei', 'microether', 'milliether', 'ether'

    Args:
        transfers: dataframe you want to convert the value of
        unit: unit to convert to

    Returns:
        a new dataframe with converted value

    Raises:
        ValueError if unit is not valid
    '''
    if unit not in ['kwei', 'mwei', 'gwei', 'microether', 'milliether', 'ether']:
        raise ValueError("Invalid unit")
    values_in_ether = transfers['value'].apply(lambda x: float(Web3.fromWei(int(x), unit)))
    transfers['value'] = values_in_ether
    return transfers


def load_transfer_data(start_date: date, end_date: date, local_path: string) -> pd.DataFrame:
    """Loads NFT token transfer data with value in ether from start_date to end_date

    Args:
        start_date: an inclusive start date for nft token transfers
        end_date: an inclusive end date for nft token transfers
        local_path: a local path where the nft toke transfer date is located
    
    Returns:
        concatenated transfers dataframe for [start_date, end_date]

    Raises:
        ValueError: if start_date is greater than end_date
        FileNotFoundError: if local_path does not exist or if no csv file is found in the local_path
    """
    if not os.path.exists(local_path):
        raise FileNotFoundError("The local_path does not exist")
    if start_date > end_date:
        raise ValueError("The 'end_date' should be equal to or greather than 'start_date'")

    transfers_list = []
    for date_ in iterate_dates(start_date, end_date):
        fname = os.path.join(local_path, "nft_token_transfers={}.csv".format(date_.strftime("%Y-%m-%d")))
        if os.path.exists(fname):
            transfers = pd.read_csv(fname, low_memory=False)
            transfers = convert_value_from_wei(transfers, 'ether')
            transfers_list.append(transfers)
        else:
            logging.error(f"{fname} does not exist")
    
    if transfers_list:
        return pd.concat(transfers_list, ignore_index=True)
    else:
        raise FileNotFoundError(f"No csv file was loaded from {local_path}")

In [4]:
start_date = date(2022,3,24)
end_date = date(2022,9,24)
local_path = "/Users/keonyonglee/Projects/nimble/nft-recommendation-data-analysis-data/nft-token-transfers"
download_nft_transfers(start_date, end_date, local_path)

In [11]:
%%time
transfers = load_transfer_data(start_date, end_date, local_path)

CPU times: user 4min 56s, sys: 29.6 s, total: 5min 25s
Wall time: 5min 30s


In [23]:
%%time
transfers.to_csv(f"{local_path}/6_months_raw_transfers.csv")

CPU times: user 7min 28s, sys: 43.4 s, total: 8min 12s
Wall time: 8min 20s


# Preprocess
- Filter out ERC1155 and ERC 165
- Filter out token_id == N/A
- Create nft_id column
- Timestamp to datetime

In [24]:
transfers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83382481 entries, 0 to 83382480
Data columns (total 17 columns):
 #   Column             Dtype  
---  ------             -----  
 0   amount             object 
 1   block_hash         object 
 2   block_number       int64  
 3   block_timestamp    object 
 4   contract_type      object 
 5   from_address       object 
 6   log_index          float64
 7   operator           object 
 8   to_address         object 
 9   token_address      object 
 10  token_id           object 
 11  transaction_hash   object 
 12  transaction_index  float64
 13  transaction_type   object 
 14  value              float64
 15  verified           int64  
 16  is_batch           bool   
dtypes: bool(1), float64(3), int64(2), object(11)
memory usage: 10.0+ GB


In [26]:
transfers.isna().sum()

amount                      0
block_hash                  0
block_number                0
block_timestamp             0
contract_type           25553
from_address                0
log_index                6966
operator             75576788
to_address                  0
token_address               0
token_id                 3644
transaction_hash            0
transaction_index       18738
transaction_type            0
value                       0
verified                    0
is_batch                    0
dtype: int64

In [12]:
%%time
transfers.fillna("N/A", inplace=True)
transfers = transfers[~transfers.contract_type.isin(['ERC1155', 'ERC165'])]
transfers = transfers[transfers.token_id != "N/A"]
transfers['nft_id'] = transfers['token_address'].str.cat(transfers['token_id'], sep="/")
transfers['block_timestamp'] = pd.to_datetime(transfers.block_timestamp)

CPU times: user 2min 14s, sys: 1min 28s, total: 3min 43s
Wall time: 4min 17s


#### Get Null Addresses List

In [13]:
genesis_addresses = [
    "0x0000000000000000000000000000000000000002",
    "0x0000000000000000000000000000000000000003",
    "0x0000000000000000000000000000000000000004",
    "0x0000000000000000000000000000000000000005",
    "0x0000000000000000000000000000000000000006",
    "0x0000000000000000000000000000000000000007",
    "0x0000000000000000000000000000000000000008",
    "0x0000000000000000000000000000000000000009",
    "0x0000000000000000000000000000000000000000",
    "0x3333333333333333333333333333333333333333",
    "0x4444444444444444444444444444444444444444",
    "0x8888888888888888888888888888888888888888",
]

In [14]:
burn_addresses = [
    "0x000000000000000000000000000000000000cdad",
    "0x000000000000000000000000000000000000dead",
    "0x0000000000000000000000000000000000000001",
    "0x0000000000000000000000000000000000000002",
    "0x0000000000000000000000000000000000000003",
    "0x0000000000000000000000000000000000000004",
    "0x0000000000000000000000000000000000000005",
    "0x0000000000000000000000000000000000000006",
    "0x0000000000000000000000000000000000000007",
    "0x0000000000000000000000000000000000000008",
    "0x0000000000000000000000000000000000000009",
    "0x0000000000000000000000000000000000000000",
    "0x00000000000000000000045261d4ee77acdb3286",
    "0x0123456789012345678901234567890123456789",
    "0x1111111111111111111111111111111111111111",
    "0x1234567890123456789012345678901234567890",
    "0x2222222222222222222222222222222222222222",
    "0x3333333333333333333333333333333333333333",
    "0x4444444444444444444444444444444444444444",
    "0x6666666666666666666666666666666666666666",
    "0x8888888888888888888888888888888888888888",
    "0xaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa",
    "0xbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbbb",
    "0xeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee",
    "0xffffffffffffffffffffffffffffffffffffffff",
    "0xdead000000000000000042069420694206942069",
]

In [15]:
null_addresses = list(set(genesis_addresses + burn_addresses))

# Badge Modeling EDA

### 1. NFT purchases per wallet address

In [188]:
%%time
nft_purchases = transfers[(~transfers.to_address.isin(null_addresses)) & (transfers.value != 0)]\
                    .rename(columns={'to_address': 'wallet_address'})\
                    .set_index(['wallet_address', 'nft_id'])['block_timestamp']['token_address']\
                    .to_frame()\
                    .rename(columns={'block_timestamp': 'purchase_time'})

CPU times: user 59.5 s, sys: 1min 6s, total: 2min 6s
Wall time: 3min 19s


In [189]:
nft_purchases

,,purchase_time
wallet_address,nft_id,
0x495f9c1ef0774bb84dd5227bc447907ba7858f17,0x29652c2e9d3656434bc8133c69258c8d05290f41/3836,2022-03-24 23:59:53+00:00
0x87b9dffd5655100fc0dd4783668baf881c2e446d,0x0144b7e66993c6bfab85581e8601f96bfe50c9df/3260,2022-03-24 23:59:43+00:00
0xfa0a1649dcd46b5eda63f951743df954a1dea952,0xbb00b6b675b9a8db9aa3b68bf0aac5e25f901656/5727,2022-03-24 23:59:43+00:00
0xb0c26305d4f4ef63f97b7d58010bbff3ce9b36b3,0x5af0d9827e0c53e4799bb226655a1de152a425a5/9331,2022-03-24 23:59:43+00:00
0x5f3ad1b031b54b15587d6cfbfc70ad20c40b7468,0xb18380485f7ba9c23deb729bedd3a3499dbd4449/6971,2022-03-24 23:59:43+00:00
...,...,...
0x12523d7753bb3d74d2900ef3af384c14d7f494a6,0xe6408dd80ee4d7596151ead35b37bdf84d2a3b0f/2287,2022-09-24 00:00:11+00:00
0xeb6d919f8421783053c0944498a434e75d20653d,0xd896b402a848331e347909a03f286ce218706f43/573,2022-09-24 00:00:11+00:00
0x07035d0e0cfb5e89218be943507694526a4ebe54,0x8f14778feb8cc049e227d01a2417a888dabb2abc/151,2022-09-24 00:00:11+00:00


#### Num of purchases per (Wallet address, NFT)
- There are around less than .4% cases where a wallet purchased an NFT multiple times

In [191]:
%%time
n_unique_nft_purchases = nft_purchases.groupby(['wallet_address', 'nft_id'])\
                            .purchase_time.nunique()\
                            .to_frame()\
                            .rename(columns={'purchase_time': 'purchase_count'})

CPU times: user 18.4 s, sys: 3.92 s, total: 22.3 s
Wall time: 23.5 s


In [192]:
n_unique_nft_purchases.describe()

,purchase_count
count,2.523694e+07
mean,1.015364e+00
std,1.094081e+00
min,1.000000e+00
25%,1.000000e+00
50%,1.000000e+00
75%,1.000000e+00
max,9.040000e+02


In [193]:
n_unique_nft_purchases[n_unique_nft_purchases.purchase_count > 1]

purchase_count
wallet_address                             nft_id                                                            
0x000000000000660def84e69995117c0176ba446e 0x7ea3cca10668b8346aec0bf1844a49e995527c8b/14892                 2
                                           0xa78c124b4f7368adde6a74d32ed9c369fe016f20/562                   2
0x00000000034b55ebd82cde9b38a85ab0978b7a47 0xca7ca7bcc765f77339be2d648ba53ce9c8a262bd/15263                 2
                                           0xca7ca7bcc765f77339be2d648ba53ce9c8a262bd/2555                  2
0x0000000009cb38fb8a1bbb8ada23c8261118f019 0x582048c4077a34e7c3799962f1f8c5342a3f4b12/5952                  2
...                                                                                                       ...
0xfffab526cb7e7aa0757a238622266493b2f7fa4a 0x57f1887a8bf19b14fc0df6fd9b2acc9af147ea85/3778...               5
0xffffe388e1e4cfaab94f0b883d28b8a424cb45a1 0x34d85c9cdeb23fa97cb08333b511ac86e1c4e258/12849                 5
                                           0x34d85c9cdeb23fa97cb08333b511ac86e1c4e258/20231                 4
                                           0xaefe0df510b172b6111bbeb65dd55b9c2ffb811a/325                   5
                                           0xaefe0df510b172b6111bbeb65dd55b9c2ffb811a/353                   2

[97817 rows x 1 columns]

In [195]:
%%time
last_nft_purchases = nft_purchases.sort_values(by='purchase_time', ascending=False)\
                        .groupby(['wallet_address', 'nft_id'])\
                        .head(1)

CPU times: user 12.3 s, sys: 10.1 s, total: 22.4 s
Wall time: 25.8 s


### 2. NFT sales per wallet address

In [200]:
%%time
nft_sales = transfers[(~transfers.from_address.isin(null_addresses)) & (transfers.value != 0)]\
                    .rename(columns={'from_address': 'wallet_address'})\
                    .set_index(['wallet_address', 'nft_id'])['block_timestamp']\
                    .to_frame()\
                    .rename(columns={'block_timestamp': 'sale_time'})

CPU times: user 35.4 s, sys: 46.8 s, total: 1min 22s
Wall time: 1min 42s


In [201]:
nft_sales

,,sale_time
wallet_address,nft_id,
0x1f8cef0cdde793f1cccc54214b00295d37781654,0x29652c2e9d3656434bc8133c69258c8d05290f41/3836,2022-03-24 23:59:53+00:00
0xaa43b3ee536455939ac6155993351a9b34f72ced,0x0144b7e66993c6bfab85581e8601f96bfe50c9df/3260,2022-03-24 23:59:43+00:00
0x7a9dc8eeaf5022cecd60c54a042343484ce6c065,0xbb00b6b675b9a8db9aa3b68bf0aac5e25f901656/5727,2022-03-24 23:59:43+00:00
0x17d86644b5aa3365c9af612c020db9947111ac99,0x5af0d9827e0c53e4799bb226655a1de152a425a5/9331,2022-03-24 23:59:43+00:00
0x08924f908484ea57efe132c0dba1924cd1b9ee7e,0xb18380485f7ba9c23deb729bedd3a3499dbd4449/6971,2022-03-24 23:59:43+00:00
...,...,...
0x283af0b28c62c092c9727f1ee09c02ca627eb7f5,0x57f1887a8bf19b14fc0df6fd9b2acc9af147ea85/103765779402199738428384986441312970450011981095550495328150325720696638752638,2022-09-24 00:00:11+00:00
0xeea477ae1312bb5bc00ef3d9d1f8da8c69e2e9da,0xe6408dd80ee4d7596151ead35b37bdf84d2a3b0f/2287,2022-09-24 00:00:11+00:00
0x0b579d10fc3cfa5b592ffc503751ddba359b2e6d,0xd896b402a848331e347909a03f286ce218706f43/573,2022-09-24 00:00:11+00:00


In [202]:
%%time
last_nft_sales = nft_sales.sort_values(by='sale_time', ascending=False)\
                    .groupby(['wallet_address', 'nft_id'])\
                    .head(1)

CPU times: user 8.68 s, sys: 2.86 s, total: 11.5 s
Wall time: 14.3 s


In [203]:
last_nft_sales

sale_time
wallet_address                             nft_id                                                                   
0xa32c8e452509b4710e85f66e70ffad5031a3a767 0x3b14d194c8cf46402beb9820dc218a15e7b0a38f/6367 2022-09-24 23:59:59+00:00
0xcd1faa80b3292e16347f31590018446552c6708f 0x0a9f2d58ba4f3a2f540275262552e6da70b0d874/3006 2022-09-24 23:59:59+00:00
0xae9c73fd0fd237c1c6f66fe009d24ce969e98704 0xf13f29330dca76be26a6c7e268da836aef978e11/232  2022-09-24 23:59:59+00:00
0x16edf13f699aabb1315fca13b1ca4e8153b92785 0xf13f29330dca76be26a6c7e268da836aef978e11/232  2022-09-24 23:59:59+00:00
0x3d5465d9dc50972cc091863299057dc70a7f32ec 0xf13f29330dca76be26a6c7e268da836aef978e11/886  2022-09-24 23:59:47+00:00
...                                                                                                              ...
0x0ec88a8b2973b21e38f8c46a6cafade2514df73c 0x3b5c947cf2858942ca471a68ddc5e048c3dbc973/491  2022-03-24 00:00:59+00:00
0x6e5d32a739ae37dd875ec028385b600ca86875b1 0x23a5ddd62aac108d1e1a81aa2b83a59055963e9e/4301 2022-03-24 00:00:36+00:00
                                           0x23a5ddd62aac108d1e1a81aa2b83a59055963e9e/444  2022-03-24 00:00:36+00:00
                                           0x23a5ddd62aac108d1e1a81aa2b83a59055963e9e/3816 2022-03-24 00:00:36+00:00
0xa5fb4870630bcfb5c4c16f7ccdd9f57d10d02eca 0x7cae7b9b9a235d1d94102598e1f23310a0618914/612  2022-03-24 00:00:36+00:00

[14355142 rows x 1 columns]

### 3. Join purchase and sales tables

In [205]:
%%time
last_nft_pns = last_nft_purchases.join(last_nft_sales, how='outer')

CPU times: user 4min 24s, sys: 9min 41s, total: 14min 6s
Wall time: 18min 24s


In [206]:
last_nft_pns

purchase_time  \
wallet_address                             nft_id                                                                         
0x000000000000000000000000000000000000dddd 0x57f1887a8bf19b14fc0df6fd9b2acc9af147ea85/1137...                       NaT   
0x0000000000000000000000000000149c7baed78b 0xc97dc59e15648b1ce35538de01990c820cb5c74e/105     2022-09-19 22:49:59+00:00   
0x000000000000000000000000249641168fc8023b 0x57f1887a8bf19b14fc0df6fd9b2acc9af147ea85/1897...                       NaT   
0x00000000000000000000005cda7ec9514b4f5959 0x85d24472b5f42bf68b870885b8c9ec58b4a1b3a4/509     2022-09-21 07:16:47+00:00   
0x0000000000000d9054f605ca65a2647c2b521422 0x459ea67815b4720e55ec7dfd93687c9d2924eb79/1047    2022-05-24 17:32:44+00:00   
...                                                                                                                 ...   
0xffffffff7a5cb56bbccb9dfaff17bb2b26a3d71c 0xe51aac67b09eaed6d3d43e794d6bae679cbe09d8/9581    2022-07-10 05:49:05+00:00   
                                           0xe51aac67b09eaed6d3d43e794d6bae679cbe09d8/9681    2022-07-10 05:49:05+00:00   
                                           0xe51aac67b09eaed6d3d43e794d6bae679cbe09d8/9948                          NaT   
                                           0xeda3b617646b5fc8c9c696e0356390128ce900f8/5705    2022-07-14 15:47:34+00:00   
                                           0xeda3b617646b5fc8c9c696e0356390128ce900f8/6248    2022-07-14 15:47:34+00:00   

                                                                                                              sale_time  
wallet_address                             nft_id                                                                        
0x000000000000000000000000000000000000dddd 0x57f1887a8bf19b14fc0df6fd9b2acc9af147ea85/1137... 2022-04-04 14:49:33+00:00  
0x0000000000000000000000000000149c7baed78b 0xc97dc59e15648b1ce35538de01990c820cb5c74e/105                           NaT  
0x000000000000000000000000249641168fc8023b 0x57f1887a8bf19b14fc0df6fd9b2acc9af147ea85/1897... 2022-09-10 17:11:44+00:00  
0x00000000000000000000005cda7ec9514b4f5959 0x85d24472b5f42bf68b870885b8c9ec58b4a1b3a4/509                           NaT  
0x0000000000000d9054f605ca65a2647c2b521422 0x459ea67815b4720e55ec7dfd93687c9d2924eb79/1047                          NaT  
...                                                                                                                 ...  
0xffffffff7a5cb56bbccb9dfaff17bb2b26a3d71c 0xe51aac67b09eaed6d3d43e794d6bae679cbe09d8/9581                          NaT  
                                           0xe51aac67b09eaed6d3d43e794d6bae679cbe09d8/9681                          NaT  
                                           0xe51aac67b09eaed6d3d43e794d6bae679cbe09d8/9948    2022-06-25 13:27:10+00:00  
                                           0xeda3b617646b5fc8c9c696e0356390128ce900f8/5705                          NaT  
                                           0xeda3b617646b5fc8c9c696e0356390128ce900f8/6248    2022-09-05 20:23:11+00:00  

[31769951 rows x 2 columns]

In [207]:
last_nft_pns.isna().sum()

purchase_time     6533009
sale_time        17414809
dtype: int64

#### Experiment with an example to learn about the cases

In [209]:
example = last_nft_pns.loc['0xffffffff7a5cb56bbccb9dfaff17bb2b26a3d71c']

In [212]:
example

,purchase_time,sale_time
nft_id,,
0x018befb7d1f3e84948466ef15cc46baf9ba5295f/2334,2022-05-25 06:06:00+00:00,NaT
0x018befb7d1f3e84948466ef15cc46baf9ba5295f/3478,2022-05-25 06:09:13+00:00,NaT
0x020cdc4775366ae436f13a7d333143432e884934/9590,2022-08-29 23:30:55+00:00,2022-09-20 13:50:23+00:00
0x07f4d0691ee248b46fb71afa15f28a08d951a002/2236,2022-07-15 05:08:18+00:00,NaT
0x07f4d0691ee248b46fb71afa15f28a08d951a002/2366,2022-07-17 16:01:45+00:00,NaT
...,...,...
0xe51aac67b09eaed6d3d43e794d6bae679cbe09d8/9581,2022-07-10 05:49:05+00:00,NaT
0xe51aac67b09eaed6d3d43e794d6bae679cbe09d8/9681,2022-07-10 05:49:05+00:00,NaT
0xe51aac67b09eaed6d3d43e794d6bae679cbe09d8/9948,NaT,2022-06-25 13:27:10+00:00


In [211]:
example[~example.purchase_time.isna() & ~example.sale_time.isna()]

,purchase_time,sale_time
nft_id,,
0x020cdc4775366ae436f13a7d333143432e884934/9590,2022-08-29 23:30:55+00:00,2022-09-20 13:50:23+00:00
0x1afef6b252cc35ec061efe6a9676c90915a73f18/1829,2022-09-08 09:35:49+00:00,2022-09-08 12:00:09+00:00
0x1afef6b252cc35ec061efe6a9676c90915a73f18/2661,2022-05-22 13:56:12+00:00,2022-05-22 17:52:46+00:00
0x1afef6b252cc35ec061efe6a9676c90915a73f18/3229,2022-06-18 09:34:41+00:00,2022-06-22 07:04:37+00:00
0x1afef6b252cc35ec061efe6a9676c90915a73f18/3384,2022-06-18 09:34:41+00:00,2022-06-21 19:17:54+00:00
0x1afef6b252cc35ec061efe6a9676c90915a73f18/4481,2022-06-18 09:34:41+00:00,2022-06-22 07:24:33+00:00
0x1afef6b252cc35ec061efe6a9676c90915a73f18/5407,2022-06-18 09:34:41+00:00,2022-09-11 17:09:28+00:00
0x1afef6b252cc35ec061efe6a9676c90915a73f18/5915,2022-06-18 09:34:41+00:00,2022-07-15 14:23:26+00:00
0x1afef6b252cc35ec061efe6a9676c90915a73f18/6214,2022-06-18 09:34:41+00:00,2022-09-09 20:30:08+00:00


In [213]:
example[example.purchase_time.isna()]

,purchase_time,sale_time
nft_id,,
0x1afef6b252cc35ec061efe6a9676c90915a73f18/5104,NaT,2022-07-15 07:27:19+00:00
0x1afef6b252cc35ec061efe6a9676c90915a73f18/5412,NaT,2022-07-15 05:59:33+00:00
0x86fc6f6c6702cef7d3bae87ef41256715416db71/1022,NaT,2022-09-13 08:08:29+00:00
0x86fc6f6c6702cef7d3bae87ef41256715416db71/1052,NaT,2022-09-13 19:23:48+00:00
0x86fc6f6c6702cef7d3bae87ef41256715416db71/145,NaT,2022-09-14 14:50:45+00:00
0x86fc6f6c6702cef7d3bae87ef41256715416db71/177,NaT,2022-09-14 19:01:15+00:00
0x86fc6f6c6702cef7d3bae87ef41256715416db71/280,NaT,2022-09-14 14:50:45+00:00
0x86fc6f6c6702cef7d3bae87ef41256715416db71/329,NaT,2022-09-14 19:05:44+00:00
0x86fc6f6c6702cef7d3bae87ef41256715416db71/933,NaT,2022-09-13 17:46:49+00:00


In [214]:
example[example.sale_time.isna()]

,purchase_time,sale_time
nft_id,,
0x018befb7d1f3e84948466ef15cc46baf9ba5295f/2334,2022-05-25 06:06:00+00:00,NaT
0x018befb7d1f3e84948466ef15cc46baf9ba5295f/3478,2022-05-25 06:09:13+00:00,NaT
0x07f4d0691ee248b46fb71afa15f28a08d951a002/2236,2022-07-15 05:08:18+00:00,NaT
0x07f4d0691ee248b46fb71afa15f28a08d951a002/2366,2022-07-17 16:01:45+00:00,NaT
0x07f4d0691ee248b46fb71afa15f28a08d951a002/2940,2022-07-14 22:35:42+00:00,NaT
...,...,...
0xe51aac67b09eaed6d3d43e794d6bae679cbe09d8/802,2022-08-27 08:51:00+00:00,NaT
0xe51aac67b09eaed6d3d43e794d6bae679cbe09d8/9303,2022-07-03 08:07:10+00:00,NaT
0xe51aac67b09eaed6d3d43e794d6bae679cbe09d8/9581,2022-07-10 05:49:05+00:00,NaT


### 4. Create 'is_owned' column

In [215]:
is_owned = last_nft_pns.sale_time.isna()\
                .to_frame()\
                .rename(columns={'sale_time': 'is_owned'})

In [216]:
is_owned

is_owned
wallet_address                             nft_id                                                      
0x000000000000000000000000000000000000dddd 0x57f1887a8bf19b14fc0df6fd9b2acc9af147ea85/1137...     False
0x0000000000000000000000000000149c7baed78b 0xc97dc59e15648b1ce35538de01990c820cb5c74e/105          True
0x000000000000000000000000249641168fc8023b 0x57f1887a8bf19b14fc0df6fd9b2acc9af147ea85/1897...     False
0x00000000000000000000005cda7ec9514b4f5959 0x85d24472b5f42bf68b870885b8c9ec58b4a1b3a4/509          True
0x0000000000000d9054f605ca65a2647c2b521422 0x459ea67815b4720e55ec7dfd93687c9d2924eb79/1047         True
...                                                                                                 ...
0xffffffff7a5cb56bbccb9dfaff17bb2b26a3d71c 0xe51aac67b09eaed6d3d43e794d6bae679cbe09d8/9581         True
                                           0xe51aac67b09eaed6d3d43e794d6bae679cbe09d8/9681         True
                                           0xe51aac67b09eaed6d3d43e794d6bae679cbe09d8/9948        False
                                           0xeda3b617646b5fc8c9c696e0356390128ce900f8/5705         True
                                           0xeda3b617646b5fc8c9c696e0356390128ce900f8/6248        False

[31769951 rows x 1 columns]

### 5. Count the number of NFTs owned per wallet

In [219]:
num_nfts_owned = is_owned.groupby('wallet_address')\
                    .is_owned.sum()\
                    .to_frame()\
                    .rename(columns={'is_owned': 'num_nfts_owned'})

In [220]:
num_nfts_owned.describe()

,num_nfts_owned
count,1.834591e+06
mean,9.492475e+00
std,5.235041e+01
min,0.000000e+00
25%,1.000000e+00
50%,2.000000e+00
75%,5.000000e+00
max,1.254800e+04


In [221]:
num_nfts_owned

,num_nfts_owned
wallet_address,
0x000000000000000000000000000000000000dddd,0
0x0000000000000000000000000000149c7baed78b,1
0x000000000000000000000000249641168fc8023b,0
0x00000000000000000000005cda7ec9514b4f5959,1
0x0000000000000d9054f605ca65a2647c2b521422,10
...,...
0xffffff5ddff86355a064c2c39e419bc06d188c7c,34
0xffffffbf3bf3b043a7457e78b4a5313a58dc5d14,1
0xffffffff45cc70237c0eb04e4c77ac6299a42acd,8


In [222]:
num_nfts_owned.quantile(.9)

num_nfts_owned    17.0
Name: 0.9, dtype: float64

### 6. Analyze top NFT collectors in terms of # of NFTs owned

In [224]:
nft_collectors = num_nfts_owned[num_nfts_owned.num_nfts_owned > 17]

In [242]:
nft_collectors.sort_values('num_nfts_owned', ascending=False).head(100)

,num_nfts_owned
wallet_address,
0x6eef09b526d883f98762a7005fabd2c800dfca44,12548
0xe6cd4727a1058e6973e973986d0493f1e7975ef8,11737
0x5a2334ca1f3773b4804d13ca4c08c543d7ea68fa,10002
0xe3379f1dca8c16d295628690df5bcd4ddfe435fe,10001
0xf82560a3d911dedbac733af3215cbc369750e04d,10000
...,...
0x67ddb2184f3429359dca8c9f9166a5020bb1603f,2197
0xcdb3c2c1c0f4f8ab403bdf4c396f5a9e4b4f2dba,2189
0x60083411d9f5fa98efb6d33d7001656f00d1cc4c,2179


#### Quick eyeball evaluation (etherscan + opensea) tells me that this needs more refinement. Especially found many cases with DeFi tokens. For those types of wallet addresses, the purchases were concentrated to few collectrions, while actual NFT collectors traded multiple collections. Let us also filter by the variety of collections.

### 7. Analyze the variety of collections for the top NFT collectors

In [239]:
%%time
nft_collectors_pns = last_nft_pns.loc[nft_collectors.index]

CPU times: user 17.7 s, sys: 50 s, total: 1min 7s
Wall time: 1min 9s


In [249]:
nft_collectors_pns = nft_collectors_pns.reset_index()

In [250]:
nft_collectors_pns

,wallet_address,nft_id,purchase_time,sale_time
0,0x000000000000b396a7ab58af828f99d4ff9ea749,0x02beed1404c69e62b76af6dbdae41bd98bca2eab/1509,NaT,2022-09-17 12:17:35+00:00
1,0x000000000000b396a7ab58af828f99d4ff9ea749,0x03235b597a78eabcb08ffcb4d97411073211dbcb/3153,2022-06-17 15:51:34+00:00,NaT
2,0x000000000000b396a7ab58af828f99d4ff9ea749,0x03235b597a78eabcb08ffcb4d97411073211dbcb/3154,2022-06-17 15:51:34+00:00,NaT
3,0x000000000000b396a7ab58af828f99d4ff9ea749,0x03235b597a78eabcb08ffcb4d97411073211dbcb/3155,2022-06-17 15:51:34+00:00,NaT
4,0x000000000000b396a7ab58af828f99d4ff9ea749,0x03235b597a78eabcb08ffcb4d97411073211dbcb/3156,2022-06-17 15:51:34+00:00,NaT
...,...,...,...,...
20779145,0xffffffff7a5cb56bbccb9dfaff17bb2b26a3d71c,0xe51aac67b09eaed6d3d43e794d6bae679cbe09d8/9581,2022-07-10 05:49:05+00:00,NaT
20779146,0xffffffff7a5cb56bbccb9dfaff17bb2b26a3d71c,0xe51aac67b09eaed6d3d43e794d6bae679cbe09d8/9681,2022-07-10 05:49:05+00:00,NaT
20779147,0xffffffff7a5cb56bbccb9dfaff17bb2b26a3d71c,0xe51aac67b09eaed6d3d43e794d6bae679cbe09d8/9948,NaT,2022-06-25 13:27:10+00:00
20779148,0xffffffff7a5cb56bbccb9dfaff17bb2b26a3d71c,0xeda3b617646b5fc8c9c696e0356390128ce900f8/5705,2022-07-14 15:47:34+00:00,NaT


In [259]:
%%time
nft_collectors_pns['collection_id'] = nft_collectors_pns.nft_id.str.split('/').str[0]

CPU times: user 21.9 s, sys: 25.8 s, total: 47.7 s
Wall time: 58.1 s


In [262]:
n_unique_collections = nft_collectors_pns.groupby('wallet_address').collection_id.nunique()

In [263]:
n_unique_collections.describe()

count    182920.000000
mean         26.095654
std          41.212303
min           1.000000
25%           6.000000
50%          15.000000
75%          31.000000
max        6474.000000
Name: collection_id, dtype: float64

### 8. Evaluate top collectors in terms of num of NFTs owned and variety of collections owned

In [761]:
for address in n_unique_collections.sort_values(ascending=False).head(200).sample(5).index:
    print(f'https://opensea.io/{address}')
    print(f'https://etherscan.io/address/{address}')

https://opensea.io/0xab1b9521de0f0a30c43817c66c54c06a95548058
https://etherscan.io/address/0xab1b9521de0f0a30c43817c66c54c06a95548058
https://opensea.io/0x76da715b266323f4eb9c9ade2127e0611f9f6c30
https://etherscan.io/address/0x76da715b266323f4eb9c9ade2127e0611f9f6c30
https://opensea.io/0xb26a76fb5da1a3cd337bc11be8b0222d2ab16e91
https://etherscan.io/address/0xb26a76fb5da1a3cd337bc11be8b0222d2ab16e91
https://opensea.io/0xffdef5e9eeaceac0800a2f784105105dbaf1d884
https://etherscan.io/address/0xffdef5e9eeaceac0800a2f784105105dbaf1d884
https://opensea.io/0x634ffd24513c0def2127e2d086a81968f948c7d7
https://etherscan.io/address/0x634ffd24513c0def2127e2d086a81968f948c7d7


### Much better!!!

# NFT Whales

#### Get non-batch sales

In [17]:
%%time
non_batch_sales = transfers[(transfers.is_batch == False) & (transfers.value > 0)]

CPU times: user 10.3 s, sys: 41.4 s, total: 51.7 s
Wall time: 1min 8s


#### Get NFT purchases per wallet

In [129]:
%%time
nft_purchases = non_batch_sales[~non_batch_sales.to_address.isin(null_addresses)]\
                    [['to_address', 'block_timestamp', 'token_address', 'value', 'nft_id']]\
                    .rename(columns={'to_address': 'wallet_address',
                                     'block_timestamp': 'purchase_time', 
                                     'token_address': 'collection_id', 
                                     'value': 'purchase_value'})\
                    .set_index(['wallet_address', 'nft_id'])

CPU times: user 24.8 s, sys: 15.6 s, total: 40.4 s
Wall time: 47.7 s


#### Filter out duplicate purchases by wallets

In [130]:
%%time
nft_purchases = nft_purchases[~nft_purchases.index.duplicated(keep='first')]

CPU times: user 840 ms, sys: 351 ms, total: 1.19 s
Wall time: 1.25 s


#### Get NFT sales per wallet

In [131]:
%%time
nft_sales = non_batch_sales[~non_batch_sales.from_address.isin(null_addresses)]\
                    [['from_address', 'block_timestamp', 'value', 'nft_id']]\
                    .rename(columns={'from_address': 'wallet_address',
                                     'block_timestamp': 'sale_time',
                                     'value': 'sale_value'})\
                    .set_index(['wallet_address', 'nft_id'])
nft_sales = nft_sales[~nft_sales.index.duplicated(keep='first')]

CPU times: user 18.3 s, sys: 14.4 s, total: 32.7 s
Wall time: 38.3 s


#### Join two tables

In [132]:
%%time
nft_pns = nft_purchases.join(nft_sales, how='outer')

CPU times: user 1min 38s, sys: 29.1 s, total: 2min 7s
Wall time: 2min 18s


In [133]:
nft_pns

purchase_time  \
wallet_address                             nft_id                                                                         
0x000000000000660def84e69995117c0176ba446e 0xc36442b4a4522e871399cd717abdd847ab11fe88/276926  2022-07-27 13:04:24+00:00   
0x0000000000006e543164be036824fcf832e67e47 0x670d4dd2e6badfbbd372d0d37e06cd2852754a04/3841                          NaT   
                                           0x670d4dd2e6badfbbd372d0d37e06cd2852754a04/3980                          NaT   
                                           0x9231f13343f871b051fa5002e8c04beed91b4f62/1315                          NaT   
                                           0xc3bac22862ff04c7b5d6d9780958d0d754b0db7a/1760... 2022-08-05 23:26:49+00:00   
...                                                                                                                 ...   
0xffffffff7a5cb56bbccb9dfaff17bb2b26a3d71c 0xe51aac67b09eaed6d3d43e794d6bae679cbe09d8/4072    2022-07-22 15:49:34+00:00   
                                           0xe51aac67b09eaed6d3d43e794d6bae679cbe09d8/7121    2022-08-21 11:34:42+00:00   
                                           0xe51aac67b09eaed6d3d43e794d6bae679cbe09d8/7911    2022-07-29 15:54:44+00:00   
                                           0xe51aac67b09eaed6d3d43e794d6bae679cbe09d8/9948                          NaT   
                                           0xeda3b617646b5fc8c9c696e0356390128ce900f8/6248                          NaT   

                                                                                                                            collection_id  \
wallet_address                             nft_id                                                                                           
0x000000000000660def84e69995117c0176ba446e 0xc36442b4a4522e871399cd717abdd847ab11fe88/276926   0xc36442b4a4522e871399cd717abdd847ab11fe88   
0x0000000000006e543164be036824fcf832e67e47 0x670d4dd2e6badfbbd372d0d37e06cd2852754a04/3841                                            NaN   
                                           0x670d4dd2e6badfbbd372d0d37e06cd2852754a04/3980                                            NaN   
                                           0x9231f13343f871b051fa5002e8c04beed91b4f62/1315                                            NaN   
                                           0xc3bac22862ff04c7b5d6d9780958d0d754b0db7a/1760...  0xc3bac22862ff04c7b5d6d9780958d0d754b0db7a   
...                                                                                                                                   ...   
0xffffffff7a5cb56bbccb9dfaff17bb2b26a3d71c 0xe51aac67b09eaed6d3d43e794d6bae679cbe09d8/4072     0xe51aac67b09eaed6d3d43e794d6bae679cbe09d8   
                                           0xe51aac67b09eaed6d3d43e794d6bae679cbe09d8/7121     0xe51aac67b09eaed6d3d43e794d6bae679cbe09d8   
                                           0xe51aac67b09eaed6d3d43e794d6bae679cbe09d8/7911     0xe51aac67b09eaed6d3d43e794d6bae679cbe09d8   
                                           0xe51aac67b09eaed6d3d43e794d6bae679cbe09d8/9948                                            NaN   
                                           0xeda3b617646b5fc8c9c696e0356390128ce900f8/6248                                            NaN   

                                                                                               purchase_value  \
wallet_address                             nft_id                                                               
0x000000000000660def84e69995117c0176ba446e 0xc36442b4a4522e871399cd717abdd847ab11fe88/276926     5.410000e-16   
0x0000000000006e543164be036824fcf832e67e47 0x670d4dd2e6badfbbd372d0d37e06cd2852754a04/3841                NaN   
                                           0x670d4dd2e6badfbbd372d0d37e06cd2852754a04/3980                NaN   
                                           0x9231f13343f871b051fa5002e8c04beed91b4f62/1315                N

In [134]:
nft_pns.isna().sum()

purchase_time     5345316
collection_id     5345316
purchase_value    5345316
sale_time         7264635
sale_value        7264635
dtype: int64

#### Add hold_time = sale_time - purchase_time

In [135]:
nft_pns['hold_time'] = nft_pns['sale_time'] - nft_pns['purchase_time']

#### Filter out purchase_time > sale_time

In [136]:
nft_pns = nft_pns[(nft_pns.hold_time.isna()) | (nft_pns.hold_time > pd.Timedelta(0,'s'))]

#### Add is_owned = sale_time.isna() (i.e. purchased but never sold)

In [137]:
%%time
is_owned = nft_pns.sale_time.isna()\
                .to_frame()\
                .rename(columns={'sale_time': 'is_owned'})
nft_pns = nft_pns.join(is_owned, how='outer')

CPU times: user 3.68 s, sys: 3.72 s, total: 7.4 s
Wall time: 8.48 s


In [138]:
nft_pns

purchase_time  \
wallet_address                             nft_id                                                                         
0x000000000000660def84e69995117c0176ba446e 0xc36442b4a4522e871399cd717abdd847ab11fe88/276926  2022-07-27 13:04:24+00:00   
0x0000000000006e543164be036824fcf832e67e47 0x670d4dd2e6badfbbd372d0d37e06cd2852754a04/3841                          NaT   
                                           0x670d4dd2e6badfbbd372d0d37e06cd2852754a04/3980                          NaT   
                                           0x9231f13343f871b051fa5002e8c04beed91b4f62/1315                          NaT   
                                           0xc3bac22862ff04c7b5d6d9780958d0d754b0db7a/1760... 2022-08-05 23:26:49+00:00   
...                                                                                                                 ...   
0xffffffff7a5cb56bbccb9dfaff17bb2b26a3d71c 0xe51aac67b09eaed6d3d43e794d6bae679cbe09d8/4072    2022-07-22 15:49:34+00:00   
                                           0xe51aac67b09eaed6d3d43e794d6bae679cbe09d8/7121    2022-08-21 11:34:42+00:00   
                                           0xe51aac67b09eaed6d3d43e794d6bae679cbe09d8/7911    2022-07-29 15:54:44+00:00   
                                           0xe51aac67b09eaed6d3d43e794d6bae679cbe09d8/9948                          NaT   
                                           0xeda3b617646b5fc8c9c696e0356390128ce900f8/6248                          NaT   

                                                                                                                            collection_id  \
wallet_address                             nft_id                                                                                           
0x000000000000660def84e69995117c0176ba446e 0xc36442b4a4522e871399cd717abdd847ab11fe88/276926   0xc36442b4a4522e871399cd717abdd847ab11fe88   
0x0000000000006e543164be036824fcf832e67e47 0x670d4dd2e6badfbbd372d0d37e06cd2852754a04/3841                                            NaN   
                                           0x670d4dd2e6badfbbd372d0d37e06cd2852754a04/3980                                            NaN   
                                           0x9231f13343f871b051fa5002e8c04beed91b4f62/1315                                            NaN   
                                           0xc3bac22862ff04c7b5d6d9780958d0d754b0db7a/1760...  0xc3bac22862ff04c7b5d6d9780958d0d754b0db7a   
...                                                                                                                                   ...   
0xffffffff7a5cb56bbccb9dfaff17bb2b26a3d71c 0xe51aac67b09eaed6d3d43e794d6bae679cbe09d8/4072     0xe51aac67b09eaed6d3d43e794d6bae679cbe09d8   
                                           0xe51aac67b09eaed6d3d43e794d6bae679cbe09d8/7121     0xe51aac67b09eaed6d3d43e794d6bae679cbe09d8   
                                           0xe51aac67b09eaed6d3d43e794d6bae679cbe09d8/7911     0xe51aac67b09eaed6d3d43e794d6bae679cbe09d8   
                                           0xe51aac67b09eaed6d3d43e794d6bae679cbe09d8/9948                                            NaN   
                                           0xeda3b617646b5fc8c9c696e0356390128ce900f8/6248                                            NaN   

                                                                                               purchase_value  \
wallet_address                             nft_id                                                               
0x000000000000660def84e69995117c0176ba446e 0xc36442b4a4522e871399cd717abdd847ab11fe88/276926     5.410000e-16   
0x0000000000006e543164be036824fcf832e67e47 0x670d4dd2e6badfbbd372d0d37e06cd2852754a04/3841                NaN   
                                           0x670d4dd2e6badfbbd372d0d37e06cd2852754a04/3980                NaN   
                                           0x9231f13343f871b051fa5002e8c04beed91b4f62/1315                N

In [139]:
potential_whales = nft_pns[nft_pns.is_owned == True][nft_pns.columns[:-4]]

In [140]:
potential_whales

purchase_time  \
wallet_address                             nft_id                                                                         
0x000000000000660def84e69995117c0176ba446e 0xc36442b4a4522e871399cd717abdd847ab11fe88/276926  2022-07-27 13:04:24+00:00   
0x0000000000006e543164be036824fcf832e67e47 0xc3bac22862ff04c7b5d6d9780958d0d754b0db7a/1760... 2022-08-05 23:26:49+00:00   
                                           0xc3bac22862ff04c7b5d6d9780958d0d754b0db7a/5738... 2022-08-05 23:33:55+00:00   
                                           0xef1a89cbfabe59397ffda11fc5df293e9bc5db90/2896    2022-07-06 21:14:26+00:00   
0x0000000000008cf7dcea7357333d113152cc14d7 0xb1f82a709342f898c3ab1c5795cc0923381be088/105     2022-08-11 01:58:47+00:00   
...                                                                                                                 ...   
0xffffffff7a5cb56bbccb9dfaff17bb2b26a3d71c 0xe51aac67b09eaed6d3d43e794d6bae679cbe09d8/283     2022-06-20 21:37:31+00:00   
                                           0xe51aac67b09eaed6d3d43e794d6bae679cbe09d8/3286    2022-07-30 04:21:44+00:00   
                                           0xe51aac67b09eaed6d3d43e794d6bae679cbe09d8/4072    2022-07-22 15:49:34+00:00   
                                           0xe51aac67b09eaed6d3d43e794d6bae679cbe09d8/7121    2022-08-21 11:34:42+00:00   
                                           0xe51aac67b09eaed6d3d43e794d6bae679cbe09d8/7911    2022-07-29 15:54:44+00:00   

                                                                                                                            collection_id  \
wallet_address                             nft_id                                                                                           
0x000000000000660def84e69995117c0176ba446e 0xc36442b4a4522e871399cd717abdd847ab11fe88/276926   0xc36442b4a4522e871399cd717abdd847ab11fe88   
0x0000000000006e543164be036824fcf832e67e47 0xc3bac22862ff04c7b5d6d9780958d0d754b0db7a/1760...  0xc3bac22862ff04c7b5d6d9780958d0d754b0db7a   
                                           0xc3bac22862ff04c7b5d6d9780958d0d754b0db7a/5738...  0xc3bac22862ff04c7b5d6d9780958d0d754b0db7a   
                                           0xef1a89cbfabe59397ffda11fc5df293e9bc5db90/2896     0xef1a89cbfabe59397ffda11fc5df293e9bc5db90   
0x0000000000008cf7dcea7357333d113152cc14d7 0xb1f82a709342f898c3ab1c5795cc0923381be088/105      0xb1f82a709342f898c3ab1c5795cc0923381be088   
...                                                                                                                                   ...   
0xffffffff7a5cb56bbccb9dfaff17bb2b26a3d71c 0xe51aac67b09eaed6d3d43e794d6bae679cbe09d8/283      0xe51aac67b09eaed6d3d43e794d6bae679cbe09d8   
                                           0xe51aac67b09eaed6d3d43e794d6bae679cbe09d8/3286     0xe51aac67b09eaed6d3d43e794d6bae679cbe09d8   
                                           0xe51aac67b09eaed6d3d43e794d6bae679cbe09d8/4072     0xe51aac67b09eaed6d3d43e794d6bae679cbe09d8   
                                           0xe51aac67b09eaed6d3d43e794d6bae679cbe09d8/7121     0xe51aac67b09eaed6d3d43e794d6bae679cbe09d8   
                                           0xe51aac67b09eaed6d3d43e794d6bae679cbe09d8/7911     0xe51aac67b09eaed6d3d43e794d6bae679cbe09d8   

                                                                                               purchase_value  
wallet_address                             nft_id                                                              
0x000000000000660def84e69995117c0176ba446e 0xc36442b4a4522e871399cd717abdd847ab11fe88/276926     5.410000e-16  
0x0000000000006e543164be036824fcf832e67e47 0xc3bac22862ff04c7b5d6d9780958d0d754b0db7a/1760...    6.000000e-02  
                                           0xc3bac22862ff04c7b5d6d9780958d0d754b0db7a/5738...    6.000000e-02  
                                           0xef1a89cbfabe59397ffda11fc5df293e9bc5db90/2896       7.500000e-02  


In [141]:
pv_cut = potential_whales.purchase_value.quantile(.975)

In [142]:
pv_cut

2.5

In [143]:
potential_whales['is_high_value'] = potential_whales.purchase_value > pv_cut

In [144]:
%%time
whales = potential_whales.groupby('wallet_address')\
            .is_high_value\
            .sum()\
            .to_frame()\
            .rename(columns={'is_high_value': 'num_high_value'})

CPU times: user 365 ms, sys: 834 ms, total: 1.2 s
Wall time: 1.44 s


In [145]:
%%time
whales['num_collections'] = potential_whales.groupby('wallet_address').collection_id.nunique()

CPU times: user 3 s, sys: 100 ms, total: 3.1 s
Wall time: 3.1 s


In [146]:
nc_cut = whales.num_collections.quantile(.9)

In [147]:
nc_cut

7.0

In [148]:
nhv_cut = whales[whales.num_collections > nc_cut].num_high_value.quantile(.99)

In [149]:
nhv_cut

10.0

In [150]:
whales = whales[(whales.num_collections > nc_cut) & (whales.num_high_value > nhv_cut)]

In [151]:
whales

,num_high_value,num_collections
wallet_address,,
0x002a99690ab1989b7971db28cd4a7673e1e6f495,26,18
0x005018f9716f7c617c183fc3b4dd0eb17f72edc3,17,43
0x00e484da1156202e9dd341ad7ea9c908bb919e96,12,18
0x011c23b3aadaf3d4991f3abee262a34d18e9fdb5,33,15
0x013598ea3fd28608c23ebac199c5b4218e2ade3a,12,113
...,...,...
0xfe5573c66273313034f7ff6050c54b5402553716,11,40
0xfec3e09c1f960b2673d2330c4b37914bfb5ef41d,11,37
0xff05f68968187d6059279a105d17109b76f3b8bb,11,23


In [1233]:
for address in whales.sample(10).index:
    print(f'https://opensea.io/{address}')

https://opensea.io/0x011c23b3aadaf3d4991f3abee262a34d18e9fdb5
https://opensea.io/0x9250f8149067fc068189f95e92f0eea9673f62d0
https://opensea.io/0xc15add7eb1ba708bc7189cef6a4c47200b77a52b
https://opensea.io/0x36abd0a112b5215c062a7a7125ba8cc23986bbe0
https://opensea.io/0x2debdf4427ccbcfdbc7f29d63964499a0ec184f6
https://opensea.io/0x91b0851797873bdbcd8a2a519a92fffcfde61041
https://opensea.io/0x7fa910346b653477f7546347d0aa444b94aded2d
https://opensea.io/0x20b38b163765c3a9afef81781f06cae22fef3290
https://opensea.io/0x41797b5e0398af475422f6f33f2dc81d9a24ae33
https://opensea.io/0x8ffa85a0c59cf23967eb31c060b2ca3a920276e1


In [152]:
np.savetxt(f'{local_path}/whales.csv', whales.index, delimiter=",", fmt='%s')

# NFT Collectors

In [24]:
%%time
sales = transfers[(transfers.value > 0)]

CPU times: user 2.95 s, sys: 8.82 s, total: 11.8 s
Wall time: 15.9 s


In [28]:
%%time
nft_p = sales[~sales.to_address.isin(null_addresses)]\
                    [['to_address', 'block_timestamp', 'token_address', 'nft_id']]\
                    .rename(columns={'to_address': 'wallet_address',
                                    'block_timestamp': 'purchase_time', 
                                    'token_address': 'collection_id'})\
                    .set_index(['wallet_address', 'nft_id'])

CPU times: user 53.3 s, sys: 18.1 s, total: 1min 11s
Wall time: 1min 17s


In [29]:
%%time
nft_p = nft_p[~nft_p.index.duplicated(keep='first')]

CPU times: user 2.21 s, sys: 828 ms, total: 3.03 s
Wall time: 3.21 s


In [30]:
%%time
nft_s = sales[~sales.from_address.isin(null_addresses)]\
                    [['from_address', 'block_timestamp', 'nft_id']]\
                    .rename(columns={'from_address': 'wallet_address',
                                    'block_timestamp': 'sale_time'})\
                    .set_index(['wallet_address', 'nft_id'])

CPU times: user 31.6 s, sys: 14.3 s, total: 45.9 s
Wall time: 50.7 s


In [31]:
%%time
nft_s = nft_s[~nft_s.index.duplicated(keep='first')]

CPU times: user 1.19 s, sys: 287 ms, total: 1.48 s
Wall time: 1.53 s


In [32]:
%%time
nft_ps = nft_p.join(nft_s, how='outer')

CPU times: user 3min 24s, sys: 57.5 s, total: 4min 21s
Wall time: 4min 44s


In [33]:
%%time
nft_ps['hold_time'] = nft_ps['sale_time'] - nft_ps['purchase_time']

CPU times: user 278 ms, sys: 165 ms, total: 442 ms
Wall time: 468 ms


In [34]:
%%time
nft_ps = nft_ps[(nft_ps.hold_time.isna()) | (nft_ps.hold_time > pd.Timedelta(0,'s'))]

CPU times: user 2.68 s, sys: 3.4 s, total: 6.09 s
Wall time: 7 s


In [35]:
%%time
is_owned = nft_ps.sale_time.isna()\
                .to_frame()\
                .rename(columns={'sale_time': 'is_owned'})

CPU times: user 18.9 ms, sys: 50.9 ms, total: 69.7 ms
Wall time: 70.5 ms


In [36]:
%%time
nft_ps = nft_ps.join(is_owned, how='outer')

CPU times: user 7.15 s, sys: 4.21 s, total: 11.4 s
Wall time: 13.4 s


In [37]:
nft_ps

purchase_time  \
wallet_address                             nft_id                                                                         
0x000000000000000000000000000000000000dddd 0x57f1887a8bf19b14fc0df6fd9b2acc9af147ea85/1137...                       NaT   
0x0000000000000000000000000000149c7baed78b 0xc97dc59e15648b1ce35538de01990c820cb5c74e/105     2022-09-19 22:49:59+00:00   
0x000000000000000000000000249641168fc8023b 0x57f1887a8bf19b14fc0df6fd9b2acc9af147ea85/1897...                       NaT   
0x00000000000000000000005cda7ec9514b4f5959 0x85d24472b5f42bf68b870885b8c9ec58b4a1b3a4/509     2022-09-21 07:16:47+00:00   
0x0000000000000d9054f605ca65a2647c2b521422 0x459ea67815b4720e55ec7dfd93687c9d2924eb79/1047    2022-05-24 17:32:44+00:00   
...                                                                                                                 ...   
0xffffffff7a5cb56bbccb9dfaff17bb2b26a3d71c 0xe51aac67b09eaed6d3d43e794d6bae679cbe09d8/9581    2022-07-10 05:49:05+00:00   
                                           0xe51aac67b09eaed6d3d43e794d6bae679cbe09d8/9681    2022-07-10 05:49:05+00:00   
                                           0xe51aac67b09eaed6d3d43e794d6bae679cbe09d8/9948                          NaT   
                                           0xeda3b617646b5fc8c9c696e0356390128ce900f8/5705    2022-07-14 15:47:34+00:00   
                                           0xeda3b617646b5fc8c9c696e0356390128ce900f8/6248    2022-07-14 15:47:34+00:00   

                                                                                                                            collection_id  \
wallet_address                             nft_id                                                                                           
0x000000000000000000000000000000000000dddd 0x57f1887a8bf19b14fc0df6fd9b2acc9af147ea85/1137...                                         NaN   
0x0000000000000000000000000000149c7baed78b 0xc97dc59e15648b1ce35538de01990c820cb5c74e/105      0xc97dc59e15648b1ce35538de01990c820cb5c74e   
0x000000000000000000000000249641168fc8023b 0x57f1887a8bf19b14fc0df6fd9b2acc9af147ea85/1897...                                         NaN   
0x00000000000000000000005cda7ec9514b4f5959 0x85d24472b5f42bf68b870885b8c9ec58b4a1b3a4/509      0x85d24472b5f42bf68b870885b8c9ec58b4a1b3a4   
0x0000000000000d9054f605ca65a2647c2b521422 0x459ea67815b4720e55ec7dfd93687c9d2924eb79/1047     0x459ea67815b4720e55ec7dfd93687c9d2924eb79   
...                                                                                                                                   ...   
0xffffffff7a5cb56bbccb9dfaff17bb2b26a3d71c 0xe51aac67b09eaed6d3d43e794d6bae679cbe09d8/9581     0xe51aac67b09eaed6d3d43e794d6bae679cbe09d8   
                                           0xe51aac67b09eaed6d3d43e794d6bae679cbe09d8/9681     0xe51aac67b09eaed6d3d43e794d6bae679cbe09d8   
                                           0xe51aac67b09eaed6d3d43e794d6bae679cbe09d8/9948                                            NaN   
                                           0xeda3b617646b5fc8c9c696e0356390128ce900f8/5705     0xeda3b617646b5fc8c9c696e0356390128ce900f8   
                                           0xeda3b617646b5fc8c9c696e0356390128ce900f8/6248     0xeda3b617646b5fc8c9c696e0356390128ce900f8   

                                                                                                              sale_time  \
wallet_address                             nft_id                                                                         
0x000000000000000000000000000000000000dddd 0x57f1887a8bf19b14fc0df6fd9b2acc9af147ea85/1137... 2022-04-04 14:49:33+00:00   
0x0000000000000000000000000000149c7baed78b 0xc97dc59e15648b1ce35538de01990c820cb5c74e/105                           NaT   
0x000000000000000000000000249641168fc8023b 0x57f1887a8bf19b14fc0df6fd9b2acc9af147ea85/1897... 2022-09-10 17:11:44+00:00   
0x00000000000000000000005cda7ec9514b4f5959 0x85d24472b5f4

In [38]:
potential_collectors = nft_ps[nft_ps.is_owned]

In [39]:
%%time
collection_owned = potential_collectors.groupby('wallet_address')['collection_id']\
                                        .nunique()\
                                        .to_frame()\
                                        .rename(columns={'collection_id': 'num_collections_owned'})

CPU times: user 7.69 s, sys: 983 ms, total: 8.67 s
Wall time: 8.87 s


In [911]:
%%time
nft_owned = potential_collectors.groupby('wallet_address')['is_owned']\
                                    .sum()\
                                    .to_frame()\
                                    .rename(columns={'is_owned': 'num_nfts_owned'})

CPU times: user 557 ms, sys: 68.1 ms, total: 625 ms
Wall time: 624 ms


In [912]:
nft_collectors = nft_owned.join(collection_owned, how='outer')

CPU times: user 979 ms, sys: 9.38 ms, total: 988 ms
Wall time: 984 ms


In [913]:
%%time
nc_cut = nft_collectors.num_collections_owned.quantile(.95)

CPU times: user 21.8 ms, sys: 2.86 ms, total: 24.7 ms
Wall time: 22.4 ms


In [914]:
nc_cut

12.0

In [915]:
nn_cut = nft_collectors[nft_collectors.num_collections_owned > nc_cut].num_nfts_owned.quantile(.95)

In [916]:
nn_cut

325.0

In [917]:
nft_collectors = nft_collectors[(nft_collectors.num_collections_owned > nc_cut) & 
                                (nft_collectors.num_nfts_owned > nn_cut)]

In [918]:
nft_collectors.describe()

,num_nfts_owned,num_collections_owned
count,3720.000000,3720.000000
mean,617.139516,72.309140
std,555.167757,56.495109
min,326.000000,13.000000
25%,381.000000,36.000000
50%,472.000000,60.000000
75%,650.000000,86.000000
max,12548.000000,794.000000


In [919]:
for address in nft_collectors.sample(10).index:
    print(f'https://opensea.io/{address}')

https://opensea.io/0x0c92700cfeb30e437035a55066b10c47e7badcd6
https://opensea.io/0xb7634821d85f84dc920682654513a69c11eeaabd
https://opensea.io/0x4fc204e756ef9526f3e8147dad4768bd3e657267
https://opensea.io/0xd7fe695ac806dd128a15a782e0b8cc7732d654c8
https://opensea.io/0x1f1048b6317339da814f9737ea119c009d441d25
https://opensea.io/0xc61ecc09913be00f9872ef5ae9f411dadb90519c
https://opensea.io/0x54be3a794282c030b15e43ae2bb182e14c409c5e
https://opensea.io/0xb0e8f66e4ce09c0c108bc96fa47beb2041b7dd2b
https://opensea.io/0x12df51d6cfafb4539444b28e1b86fb6b796b12a4
https://opensea.io/0x15c8949cf2f16fd26b7b070bfd583216ae944382


In [1174]:
nft_collectors.to_csv(f'{local_path}/nft_collectors.csv')

# NFT Filppers

In [40]:
%%time
potential_flippers = nft_ps[~nft_ps['hold_time'].isna()]

CPU times: user 232 ms, sys: 64 ms, total: 296 ms
Wall time: 309 ms


In [41]:
ht_cut = potential_flippers.hold_time.quantile(.2)

In [42]:
ht_cut

Timedelta('0 days 01:11:33')

In [43]:
%%time
is_flipped = (potential_flippers.hold_time  < ht_cut).to_frame().rename(columns={'hold_time': 'is_flipped'})

CPU times: user 9.24 ms, sys: 2.63 ms, total: 11.9 ms
Wall time: 11.6 ms


In [44]:
%%time
potential_flippers = potential_flippers.join(is_flipped, how='outer')

CPU times: user 571 ms, sys: 177 ms, total: 748 ms
Wall time: 771 ms


In [45]:
%%time
nft_flippers = potential_flippers.groupby('wallet_address')\
                .is_flipped.sum()\
                .to_frame()\
                .rename(columns={'is_flipped': 'num_flipped'})

CPU times: user 195 ms, sys: 46.7 ms, total: 242 ms
Wall time: 258 ms


In [46]:
nft_flippers['num_collections'] = potential_flippers.groupby('wallet_address').collection_id.nunique()

In [48]:
nc_cut = nft_flippers.num_collections.quantile(.90)

In [49]:
nc_cut

11.0

In [50]:
nf_cut = nft_flippers[nft_flippers.num_collections > nc_cut].num_flipped.quantile(.95)

In [51]:
nf_cut

63.0

In [52]:
nft_flippers = nft_flippers[(nft_flippers.num_collections > nc_cut) & (nft_flippers.num_flipped > nf_cut)]

In [53]:
nft_flippers

,num_flipped,num_collections
wallet_address,,
0x00003183f59e825911d98fb509a157cd2abbae25,176,13
0x0011a5b88073adf053ec6c602136e9b24b26b485,162,104
0x006236cc0510792b7f5bd3b0a5b0c50a764fbca9,195,53
0x007287045479ce07488b2ba6482af01754ae05ee,65,26
0x00c3ff532fee97d06b7ef8bf14db72fadb005007,109,34
...,...,...
0xff274763062ab364fe17d17cc1cc10fa9e41e040,143,141
0xff5ad07002d876bbb63a17d3f5bab8f63f034e1e,231,127
0xff747758a76e56a641f7bc9dd87f810af773fef2,217,19


In [945]:
# go to their activity to eyeball
for address in nft_flippers.sample(10).index:
    print(f'https://opensea.io/{address}')

https://opensea.io/0xf45f8c39076e2d67f4e8dfdb74b5fb0817bde010
https://opensea.io/0x2f3646ef40734ca4fe9c0201999824de14edd823
https://opensea.io/0x0a73e98d1b7790a902d501f976155b6a8964274b
https://opensea.io/0xf36222036ff16cfdd1ddbf0c1a1d9ef513ba2864
https://opensea.io/0xda78017bf05690ec6f799ae1d9ed151dc53cb16e
https://opensea.io/0x78b63080950d8ed760641738b5902f83b6a483e7
https://opensea.io/0x48a5e9562e6f2e6e2908ed96e735af254e2bca84
https://opensea.io/0x3e2af1fc75a5fc0a84d69d326c7d3e1ef7534964
https://opensea.io/0xfa35f69a6dd9ef80af40661743be519e30fbf406
https://opensea.io/0x6d3c1b048f45e008f3c205c292cdb21318865dd3


In [1173]:
nft_flippers.to_csv(f'{local_path}/nft_flippers.csv')

# Early Adopters
- Purchased multiple collections early on
- For each collection
    - Get the list of buyers and timestamp
    - Keep the first purchase only per buyer
    - Add is_early
- For each wallet
    - Get num_early

In [1005]:
%%time
collection_p = sales[~sales.to_address.isin(null_addresses)]\
                    [['from_address', 'to_address', 'token_address', 'block_timestamp']]\
                    .rename(columns={'to_address': 'wallet_address', 
                                     'token_address': 'collection_id', 
                                     'block_timestamp': 'purchase_time'})

CPU times: user 5.18 s, sys: 16.8 s, total: 21.9 s
Wall time: 27.9 s


In [1006]:
collection_p

,from_address,wallet_address,collection_id,purchase_time
4,0x1f8cef0cdde793f1cccc54214b00295d37781654,0x495f9c1ef0774bb84dd5227bc447907ba7858f17,0x29652c2e9d3656434bc8133c69258c8d05290f41,2022-03-24 23:59:53+00:00
12,0xaa43b3ee536455939ac6155993351a9b34f72ced,0x87b9dffd5655100fc0dd4783668baf881c2e446d,0x0144b7e66993c6bfab85581e8601f96bfe50c9df,2022-03-24 23:59:43+00:00
13,0x7a9dc8eeaf5022cecd60c54a042343484ce6c065,0xfa0a1649dcd46b5eda63f951743df954a1dea952,0xbb00b6b675b9a8db9aa3b68bf0aac5e25f901656,2022-03-24 23:59:43+00:00
14,0x17d86644b5aa3365c9af612c020db9947111ac99,0xb0c26305d4f4ef63f97b7d58010bbff3ce9b36b3,0x5af0d9827e0c53e4799bb226655a1de152a425a5,2022-03-24 23:59:43+00:00
15,0x08924f908484ea57efe132c0dba1924cd1b9ee7e,0x5f3ad1b031b54b15587d6cfbfc70ad20c40b7468,0xb18380485f7ba9c23deb729bedd3a3499dbd4449,2022-03-24 23:59:43+00:00
...,...,...,...,...
83382457,0xeea477ae1312bb5bc00ef3d9d1f8da8c69e2e9da,0x12523d7753bb3d74d2900ef3af384c14d7f494a6,0xe6408dd80ee4d7596151ead35b37bdf84d2a3b0f,2022-09-24 00:00:11+00:00
83382460,0x0b579d10fc3cfa5b592ffc503751ddba359b2e6d,0xeb6d919f8421783053c0944498a434e75d20653d,0xd896b402a848331e347909a03f286ce218706f43,2022-09-24 00:00:11+00:00
83382476,0x47efd37de4fe5e5ca9934ed6c3c5e2aab093372f,0x07035d0e0cfb5e89218be943507694526a4ebe54,0x8f14778feb8cc049e227d01a2417a888dabb2abc,2022-09-24 00:00:11+00:00
83382478,0x283af0b28c62c092c9727f1ee09c02ca627eb7f5,0x5192b3fe45a0a649b2709e2264d537c76190aa5c,0x57f1887a8bf19b14fc0df6fd9b2acc9af147ea85,2022-09-24 00:00:11+00:00


In [1007]:
%%time
first_cp = collection_p.sort_values(by='purchase_time', ascending=True)\
                        .groupby(['collection_id', 'wallet_address'])\
                        .head(1)

CPU times: user 11.2 s, sys: 1.01 s, total: 12.2 s
Wall time: 12.5 s


In [1009]:
minted_collections = first_cp[first_cp.from_address.isin(genesis_addresses)].collection_id

In [1011]:
mcp = first_cp[first_cp.collection_id.isin(minted_collections)]

In [1012]:
mcp

,from_address,wallet_address,collection_id,purchase_time
381833,0x0000000000000000000000000000000000000000,0x7f65353f0b0e4afddd2e47b1b1216cd2cb9b10c0,0x701a038af4bd0fc9b69a829ddcb2f61185a49568,2022-03-24 00:00:36+00:00
381806,0x0000000000000000000000000000000000000000,0x228b9de49ea45b662bd6034aa9d24e65d6e0ba1f,0x701a038af4bd0fc9b69a829ddcb2f61185a49568,2022-03-24 00:00:36+00:00
381805,0x0000000000000000000000000000000000000000,0xa377ac2b83fe87b8b712efab46e6ccb1bdf83189,0x701a038af4bd0fc9b69a829ddcb2f61185a49568,2022-03-24 00:00:36+00:00
381804,0x0000000000000000000000000000000000000000,0x47d84f6483b1ebc69008fafa726d1b303ec82ffd,0x701a038af4bd0fc9b69a829ddcb2f61185a49568,2022-03-24 00:00:36+00:00
381803,0x0000000000000000000000000000000000000000,0x654322a5f7dd26358377a97ae3157172b8d6bf5a,0x701a038af4bd0fc9b69a829ddcb2f61185a49568,2022-03-24 00:00:36+00:00
...,...,...,...,...
82755755,0x3d5465d9dc50972cc091863299057dc70a7f32ec,0x3b02563b0971c8a893bbf51533c689686676698a,0xf13f29330dca76be26a6c7e268da836aef978e11,2022-09-24 23:59:47+00:00
82755756,0x4d25f3e95ed0ae1ebd8189eae7f0de8a70519032,0x34655e77a7125bcb4fc7cd552733958828e8b2ee,0xf13f29330dca76be26a6c7e268da836aef978e11,2022-09-24 23:59:47+00:00
82755695,0xae9c73fd0fd237c1c6f66fe009d24ce969e98704,0x3f6ac3037ffe0239b39438193c6bede7305e8265,0xf13f29330dca76be26a6c7e268da836aef978e11,2022-09-24 23:59:59+00:00
82755694,0xcd1faa80b3292e16347f31590018446552c6708f,0xd760b194c7760cc66f4452833046eed5ceec3a3a,0x0a9f2d58ba4f3a2f540275262552e6da70b0d874,2022-09-24 23:59:59+00:00


In [1013]:
mcp = mcp.set_index(['collection_id', 'wallet_address'])

In [1018]:
example = mcp.loc["0x701a038af4bd0fc9b69a829ddcb2f61185a49568"]

In [1019]:
example

,from_address,purchase_time
wallet_address,,
0x7f65353f0b0e4afddd2e47b1b1216cd2cb9b10c0,0x0000000000000000000000000000000000000000,2022-03-24 00:00:36+00:00
0x228b9de49ea45b662bd6034aa9d24e65d6e0ba1f,0x0000000000000000000000000000000000000000,2022-03-24 00:00:36+00:00
0xa377ac2b83fe87b8b712efab46e6ccb1bdf83189,0x0000000000000000000000000000000000000000,2022-03-24 00:00:36+00:00
0x47d84f6483b1ebc69008fafa726d1b303ec82ffd,0x0000000000000000000000000000000000000000,2022-03-24 00:00:36+00:00
0x654322a5f7dd26358377a97ae3157172b8d6bf5a,0x0000000000000000000000000000000000000000,2022-03-24 00:00:36+00:00
...,...,...
0x1c65d30763ff5e155fa53e9310681d3508865cfc,0x6ef85df8fd2cb29dd958c94dde3644406d63d264,2022-09-22 01:45:23+00:00
0x9e18fd6402f7755048a0f1319ebaa9d9c5acf3d5,0xa5c5e686f7f4ff921a4c9bc0a512c3183cbc863b,2022-09-23 09:47:47+00:00
0xa5c5e686f7f4ff921a4c9bc0a512c3183cbc863b,0xaa986ef5ac55bacf72046e2884dc6fbfe3332f18,2022-09-23 09:47:47+00:00


In [1022]:
example[example.from_address.isin(genesis_addresses)].purchase_time.describe(datetime_is_numeric=True)

count                                   2653
mean     2022-03-24 14:35:39.915567360+00:00
min                2022-03-24 00:00:36+00:00
25%                2022-03-24 00:22:40+00:00
50%                2022-03-24 00:24:14+00:00
75%                2022-03-25 00:30:38+00:00
max                2022-03-26 01:14:15+00:00
Name: purchase_time, dtype: object

In [1023]:
example[~example.from_address.isin(genesis_addresses)].purchase_time.describe(datetime_is_numeric=True)

count                                   9905
mean     2022-04-04 09:21:06.456335104+00:00
min                2022-03-24 00:24:34+00:00
25%                2022-03-27 18:30:20+00:00
50%                2022-03-30 16:30:57+00:00
75%                2022-04-03 01:35:34+00:00
max                2022-09-24 17:51:47+00:00
Name: purchase_time, dtype: object

### Detect new collections using mint events but minting does not necessarily mean early adopter

In [54]:
%%time
minted_collections = sales[sales.from_address.isin(genesis_addresses)].token_address.unique()

CPU times: user 4.24 s, sys: 13.9 s, total: 18.1 s
Wall time: 25.5 s


In [55]:
%%time
mcp = sales[~sales.to_address.isin(null_addresses) & sales.token_address.isin(minted_collections)]\
                    [['to_address', 'token_address', 'block_timestamp']]\
                    .rename(columns={'to_address': 'wallet_address', 
                                     'token_address': 'collection_id', 
                                     'block_timestamp': 'purchase_time'})

CPU times: user 4.95 s, sys: 15.4 s, total: 20.4 s
Wall time: 26 s


In [56]:
mcp

,wallet_address,collection_id,purchase_time
12,0x87b9dffd5655100fc0dd4783668baf881c2e446d,0x0144b7e66993c6bfab85581e8601f96bfe50c9df,2022-03-24 23:59:43+00:00
15,0x5f3ad1b031b54b15587d6cfbfc70ad20c40b7468,0xb18380485f7ba9c23deb729bedd3a3499dbd4449,2022-03-24 23:59:43+00:00
16,0x9e8504058397db6f6ee73256aa0936015973a73e,0x325bad883b4e9a35277e99902d94dd18186ae219,2022-03-24 23:59:43+00:00
20,0xd4e9eb3e4d8f8c84809fb29c96914b1bd1fbbf20,0x0144b7e66993c6bfab85581e8601f96bfe50c9df,2022-03-24 23:59:43+00:00
27,0x3f1c0f21bc378120aca83120c302bdb792538586,0x57f1887a8bf19b14fc0df6fd9b2acc9af147ea85,2022-03-24 23:59:43+00:00
...,...,...,...
83382455,0x283af0b28c62c092c9727f1ee09c02ca627eb7f5,0x57f1887a8bf19b14fc0df6fd9b2acc9af147ea85,2022-09-24 00:00:11+00:00
83382460,0xeb6d919f8421783053c0944498a434e75d20653d,0xd896b402a848331e347909a03f286ce218706f43,2022-09-24 00:00:11+00:00
83382476,0x07035d0e0cfb5e89218be943507694526a4ebe54,0x8f14778feb8cc049e227d01a2417a888dabb2abc,2022-09-24 00:00:11+00:00
83382478,0x5192b3fe45a0a649b2709e2264d537c76190aa5c,0x57f1887a8bf19b14fc0df6fd9b2acc9af147ea85,2022-09-24 00:00:11+00:00


In [57]:
%%time
first_mcp = mcp.sort_values(by='purchase_time', ascending=True)\
                        .groupby(['collection_id', 'wallet_address'])\
                        .head(1)

CPU times: user 7.95 s, sys: 678 ms, total: 8.63 s
Wall time: 8.85 s


In [58]:
%%time
collections = first_mcp.groupby('collection_id').purchase_time\
                        .nunique()\
                        .to_frame()\
                        .rename(columns={'purchase_time': 'num_purchase_time'})

CPU times: user 1.05 s, sys: 45.1 ms, total: 1.09 s
Wall time: 1.1 s


In [59]:
%%time
collections['num_wallets'] = first_mcp.groupby('collection_id')['wallet_address'].nunique()

CPU times: user 3.62 s, sys: 55.1 ms, total: 3.67 s
Wall time: 3.67 s


In [60]:
collections.describe()

,num_purchase_time,num_wallets
count,21178.000000,21178.000000
mean,148.788932,244.223487
std,1520.355953,2116.623873
min,1.000000,1.000000
25%,2.000000,2.000000
50%,14.000000,16.000000
75%,63.000000,95.000000
max,204899.000000,267752.000000


### Let us consider top 50% collections in terms of num_purchase_time and num_wallets

In [61]:
npt_cut = collections.num_purchase_time.quantile(.5)
nw_cut = collections.num_wallets.quantile(.5)
top_collections = collections[(collections.num_purchase_time > npt_cut) & (collections.num_wallets > nw_cut)]

In [62]:
top_collections.describe()

,num_purchase_time,num_wallets
count,10146.000000,10146.000000
mean,305.838754,503.878671
std,2185.793084,3036.830648
min,15.000000,17.000000
25%,32.000000,41.000000
50%,68.000000,103.000000
75%,202.000000,329.000000
max,204899.000000,267752.000000


In [63]:
top_first_mcp = first_mcp[first_mcp.collection_id.isin(top_collections.index)]

In [64]:
top_first_mcp

,wallet_address,collection_id,purchase_time
381825,0xd749eb1c3d4bb2829050dded52d9228ff81ef7a6,0x701a038af4bd0fc9b69a829ddcb2f61185a49568,2022-03-24 00:00:36+00:00
381806,0x228b9de49ea45b662bd6034aa9d24e65d6e0ba1f,0x701a038af4bd0fc9b69a829ddcb2f61185a49568,2022-03-24 00:00:36+00:00
381805,0xa377ac2b83fe87b8b712efab46e6ccb1bdf83189,0x701a038af4bd0fc9b69a829ddcb2f61185a49568,2022-03-24 00:00:36+00:00
381804,0x47d84f6483b1ebc69008fafa726d1b303ec82ffd,0x701a038af4bd0fc9b69a829ddcb2f61185a49568,2022-03-24 00:00:36+00:00
381803,0x654322a5f7dd26358377a97ae3157172b8d6bf5a,0x701a038af4bd0fc9b69a829ddcb2f61185a49568,2022-03-24 00:00:36+00:00
...,...,...,...
82755756,0x34655e77a7125bcb4fc7cd552733958828e8b2ee,0xf13f29330dca76be26a6c7e268da836aef978e11,2022-09-24 23:59:47+00:00
82755755,0x3b02563b0971c8a893bbf51533c689686676698a,0xf13f29330dca76be26a6c7e268da836aef978e11,2022-09-24 23:59:47+00:00
82755695,0x3f6ac3037ffe0239b39438193c6bede7305e8265,0xf13f29330dca76be26a6c7e268da836aef978e11,2022-09-24 23:59:59+00:00
82755694,0xd760b194c7760cc66f4452833046eed5ceec3a3a,0x0a9f2d58ba4f3a2f540275262552e6da70b0d874,2022-09-24 23:59:59+00:00


In [65]:
def compute_quantile(g):
    return g.purchase_time.quantile(.10)

In [66]:
%%time
pt_cut = top_first_mcp.groupby('collection_id')\
            .apply(lambda g: compute_quantile(g))\
            .to_frame()\
            .rename(columns={0: 'purchase_time_cut'})

CPU times: user 2.5 s, sys: 54.1 ms, total: 2.55 s
Wall time: 2.55 s


In [67]:
pt_cut

,purchase_time_cut
collection_id,
0x00000003e02101799c6d82dc467db1fbcb4d867f,2022-07-08 23:18:29+00:00
0x000803758151829d103ff188581f87038ae3b893,2022-06-07 09:25:40+00:00
0x00139fdd8f2bb8b4376aab2435c44981ec34e7e0,2022-04-14 15:00:52+00:00
0x001ba0474ff8796a007afba716237fc5d2e8299e,2022-09-20 06:12:59+00:00
0x001c46908d9d42f5677e89a01fa77bfa3792580b,2022-06-12 20:59:08.400000+00:00
...,...
0xffb3599a8723a8a7dc47420e7e14687aa1d9070c,2022-05-17 17:15:39+00:00
0xffb3d244702596494f3df90ea89c26a8d962197c,2022-07-22 19:29:03+00:00
0xffba892d276e4687e44edfa9de529851559f3634,2022-05-16 17:08:21.800000+00:00


In [68]:
%%time
top_first_mcp = top_first_mcp.merge(pt_cut, how='outer', on='collection_id')

CPU times: user 414 ms, sys: 41.8 ms, total: 455 ms
Wall time: 461 ms


In [84]:
top_first_mcp

,wallet_address,collection_id,purchase_time,purchase_time_cut,is_early
0,0xd749eb1c3d4bb2829050dded52d9228ff81ef7a6,0x701a038af4bd0fc9b69a829ddcb2f61185a49568,2022-03-24 00:00:36+00:00,2022-03-24 00:24:13+00:00,True
1,0x228b9de49ea45b662bd6034aa9d24e65d6e0ba1f,0x701a038af4bd0fc9b69a829ddcb2f61185a49568,2022-03-24 00:00:36+00:00,2022-03-24 00:24:13+00:00,True
2,0xa377ac2b83fe87b8b712efab46e6ccb1bdf83189,0x701a038af4bd0fc9b69a829ddcb2f61185a49568,2022-03-24 00:00:36+00:00,2022-03-24 00:24:13+00:00,True
3,0x47d84f6483b1ebc69008fafa726d1b303ec82ffd,0x701a038af4bd0fc9b69a829ddcb2f61185a49568,2022-03-24 00:00:36+00:00,2022-03-24 00:24:13+00:00,True
4,0x654322a5f7dd26358377a97ae3157172b8d6bf5a,0x701a038af4bd0fc9b69a829ddcb2f61185a49568,2022-03-24 00:00:36+00:00,2022-03-24 00:24:13+00:00,True
...,...,...,...,...,...
5112348,0xc7daf473c103aa2b112fe2f773e3a508a6999bb6,0xd8b018794808df7d3f4fa3526369b37b359c6d5b,2022-09-24 23:44:11+00:00,2022-09-24 23:19:30.200000+00:00,False
5112349,0x4bde504b7d1e90d0e9ec3c7fa495a1e0306448b4,0xd8b018794808df7d3f4fa3526369b37b359c6d5b,2022-09-24 23:46:11+00:00,2022-09-24 23:19:30.200000+00:00,False
5112350,0x6748c23cb9d9f40ac75ec2c43106a8bc3197f82e,0xd8b018794808df7d3f4fa3526369b37b359c6d5b,2022-09-24 23:48:23+00:00,2022-09-24 23:19:30.200000+00:00,False
5112351,0xfcf951562b0ff16c8e460b73aeff8432e15f0a48,0xd8b018794808df7d3f4fa3526369b37b359c6d5b,2022-09-24 23:50:47+00:00,2022-09-24 23:19:30.200000+00:00,False


In [70]:
top_first_mcp['is_early'] = top_first_mcp['purchase_time'] < top_first_mcp['purchase_time_cut']

In [71]:
top_first_mcp

,wallet_address,collection_id,purchase_time,purchase_time_cut,is_early
0,0xd749eb1c3d4bb2829050dded52d9228ff81ef7a6,0x701a038af4bd0fc9b69a829ddcb2f61185a49568,2022-03-24 00:00:36+00:00,2022-03-24 00:24:13+00:00,True
1,0x228b9de49ea45b662bd6034aa9d24e65d6e0ba1f,0x701a038af4bd0fc9b69a829ddcb2f61185a49568,2022-03-24 00:00:36+00:00,2022-03-24 00:24:13+00:00,True
2,0xa377ac2b83fe87b8b712efab46e6ccb1bdf83189,0x701a038af4bd0fc9b69a829ddcb2f61185a49568,2022-03-24 00:00:36+00:00,2022-03-24 00:24:13+00:00,True
3,0x47d84f6483b1ebc69008fafa726d1b303ec82ffd,0x701a038af4bd0fc9b69a829ddcb2f61185a49568,2022-03-24 00:00:36+00:00,2022-03-24 00:24:13+00:00,True
4,0x654322a5f7dd26358377a97ae3157172b8d6bf5a,0x701a038af4bd0fc9b69a829ddcb2f61185a49568,2022-03-24 00:00:36+00:00,2022-03-24 00:24:13+00:00,True
...,...,...,...,...,...
5112348,0xc7daf473c103aa2b112fe2f773e3a508a6999bb6,0xd8b018794808df7d3f4fa3526369b37b359c6d5b,2022-09-24 23:44:11+00:00,2022-09-24 23:19:30.200000+00:00,False
5112349,0x4bde504b7d1e90d0e9ec3c7fa495a1e0306448b4,0xd8b018794808df7d3f4fa3526369b37b359c6d5b,2022-09-24 23:46:11+00:00,2022-09-24 23:19:30.200000+00:00,False
5112350,0x6748c23cb9d9f40ac75ec2c43106a8bc3197f82e,0xd8b018794808df7d3f4fa3526369b37b359c6d5b,2022-09-24 23:48:23+00:00,2022-09-24 23:19:30.200000+00:00,False
5112351,0xfcf951562b0ff16c8e460b73aeff8432e15f0a48,0xd8b018794808df7d3f4fa3526369b37b359c6d5b,2022-09-24 23:50:47+00:00,2022-09-24 23:19:30.200000+00:00,False


In [72]:
is_early_count = top_first_mcp.groupby('wallet_address').is_early.sum()\
                    .to_frame()\
                    .rename(columns={'is_early': 'is_early_count'})

In [73]:
is_early_count.describe()

,is_early_count
count,1.381733e+06
mean,3.522699e-01
std,1.278810e+00
min,0.000000e+00
25%,0.000000e+00
50%,0.000000e+00
75%,0.000000e+00
max,5.100000e+02


In [85]:
early_adopters = is_early_count[is_early_count.is_early_count > 10]

In [86]:
early_adopters

,is_early_count
wallet_address,
0x000000feee8780d16539eb78ad2f2702e8c3c844,11
0x000005a271a610964bb42658c7ff50fee2aa055a,11
0x001a181ab8c41045e26dd2245ffcc12818ea742f,12
0x00385f60f4b5234e96989805af7328f7afd742b1,17
0x003f35595dce3187b4fff2b5a2c4303f7158208a,15
...,...
0xff209651029de229108553c3fd3115b570327de6,13
0xff444de9dab3e057f6ad774e0b8f7a3053b49803,11
0xff9911abdbe9d1f7d1a19595b93905c2a9ad60f4,17


In [1146]:
for address in early_adopters.sample(10).index:
    print(f'https://opensea.io/{address}')

https://opensea.io/0x34f0bbb69c2e0f7d05e604d76b80e614089be386
https://opensea.io/0x40043fffc2642a4e7a467dc778deb84ed5e64e50
https://opensea.io/0x138b78bca19b534c912509b800bb70410511fd8c
https://opensea.io/0x730a2b599580492ee02cbb9cfd4c0424c67377d9
https://opensea.io/0x2a5434f33d621331552e368f8e5b685d6d055588
https://opensea.io/0x3a33e605008433f1eda7bd5c65befa9fccd8c344
https://opensea.io/0x7693c3545667309f112eb2d1a0d7bdfcfc536411
https://opensea.io/0x70ce25281d209e0acdc799b7d2ea5efb15db749c
https://opensea.io/0xc401f424fed8b111330c6915467d231997e51b9f
https://opensea.io/0x94c1e3a7d8a2cbb0ab306d05148f0f24f1e1c2bb


In [1172]:
early_adopters.to_csv(f'{local_path}/early_adopters.csv')

### Do early adopters and NFT flippers have overlap?

In [89]:
early_adopters_list = early_adopters.index.to_list()
nft_flippers_list = nft_flippers.index.to_list()
len(set(early_adopters_list).intersection(nft_flippers_list))

1041

### Add filppers filter before getting early adopters

In [91]:
%%time
mcp = sales[~sales.to_address.isin(null_addresses) 
            & sales.token_address.isin(minted_collections)
            & ~sales.to_address.isin(nft_flippers.index)]\
                    [['to_address', 'token_address', 'block_timestamp']]\
                    .rename(columns={'to_address': 'wallet_address', 
                                     'token_address': 'collection_id', 
                                     'block_timestamp': 'purchase_time'})

CPU times: user 5.45 s, sys: 14.2 s, total: 19.6 s
Wall time: 24.4 s


In [92]:
%%time
first_mcp = mcp.sort_values(by='purchase_time', ascending=True)\
                        .groupby(['collection_id', 'wallet_address'])\
                        .head(1)

CPU times: user 7.6 s, sys: 664 ms, total: 8.27 s
Wall time: 8.6 s


In [93]:
%%time
collections = first_mcp.groupby('collection_id').purchase_time\
                        .nunique()\
                        .to_frame()\
                        .rename(columns={'purchase_time': 'num_purchase_time'})
collections['num_wallets'] = first_mcp.groupby('collection_id')['wallet_address'].nunique()

CPU times: user 4.34 s, sys: 89 ms, total: 4.43 s
Wall time: 4.43 s


In [94]:
%%time
npt_cut = collections.num_purchase_time.quantile(.5)
nw_cut = collections.num_wallets.quantile(.5)
top_collections = collections[(collections.num_purchase_time > npt_cut) & (collections.num_wallets > nw_cut)]
top_first_mcp = first_mcp[first_mcp.collection_id.isin(top_collections.index)]

CPU times: user 295 ms, sys: 24 ms, total: 319 ms
Wall time: 319 ms


In [95]:
%%time
pt_cut = top_first_mcp.groupby('collection_id')\
            .apply(lambda g: compute_quantile(g))\
            .to_frame()\
            .rename(columns={0: 'purchase_time_cut'})
top_first_mcp = top_first_mcp.merge(pt_cut, how='outer', on='collection_id')
top_first_mcp['is_early'] = top_first_mcp['purchase_time'] < top_first_mcp['purchase_time_cut']

CPU times: user 2.91 s, sys: 85.8 ms, total: 2.99 s
Wall time: 3.01 s


In [96]:
is_early_count = top_first_mcp.groupby('wallet_address').is_early.sum()\
                    .to_frame()\
                    .rename(columns={'is_early': 'is_early_count'})

In [97]:
is_early_count.describe()

,is_early_count
count,1.380092e+06
mean,3.365623e-01
std,1.118867e+00
min,0.000000e+00
25%,0.000000e+00
50%,0.000000e+00
75%,0.000000e+00
max,5.210000e+02


In [118]:
is_early_count.quantile(.998)

is_early_count    9.0
Name: 0.998, dtype: float64

In [119]:
early_adopters = is_early_count[is_early_count.is_early_count > 9]

In [120]:
early_adopters

,is_early_count
wallet_address,
0x000000feee8780d16539eb78ad2f2702e8c3c844,12
0x000005a271a610964bb42658c7ff50fee2aa055a,11
0x000c987f621b3788f84112fa7a1e8b42ab8cc212,11
0x000cef8f0ab0a2fadbecaeb92e8781ab3e04b01e,10
0x001a181ab8c41045e26dd2245ffcc12818ea742f,12
...,...
0xff13cbc9f9b94f3c47c1008a52585287985c3d8f,28
0xff209651029de229108553c3fd3115b570327de6,15
0xff444de9dab3e057f6ad774e0b8f7a3053b49803,11


In [121]:
for address in early_adopters.sample(10).index:
    print(f'https://opensea.io/{address}')

https://opensea.io/0xd046c682cc3767ee363e00aa9601ce4014e46920
https://opensea.io/0x6a69131ee61d0058c9d25519ec589093a292e868
https://opensea.io/0x9fb4f3d93205f4a50512223582c91056b057a4ea
https://opensea.io/0x1e6c1375abc832540da73994bac82a2225a9bfbd
https://opensea.io/0x1cddcd54d41d45da496b2c74d3e606bb876ea54e
https://opensea.io/0x57a099bbf0932c730f78252918be5ff44d799107
https://opensea.io/0x3e3fd41add7de67a12cbf9575d442826b067c71a
https://opensea.io/0xf74d8ca88b666bd06f10614ca8ae1b8c9b43d206
https://opensea.io/0x1b2a9ffccb9757a30c2ad2473c2b5ae4380471a2
https://opensea.io/0xa410c73cec32a90ddbdbf3ab22ff577223e79f61


In [128]:
np.savetxt(f'{local_path}/early_adopters.csv', early_adopters.index, delimiter=",", fmt='%s')

# Profitable Flippers

In [3]:
%%time
local_path = "/Users/keonyonglee/Projects/nimble/nft-recommendation-data-analysis-data/nft-token-transfers"
non_batch_ps = pd.read_csv(f"{local_path}/wallet_profile.csv")

CPU times: user 18.6 s, sys: 1.75 s, total: 20.4 s
Wall time: 22.7 s


In [4]:
non_batch_ps

,wallet_address,nft_id,purchase_time,collection_id,purchase_value,sale_time,sale_value,hold_time,is_owned
0,0x000000000000660def84e69995117c0176ba446e,0xc36442b4a4522e871399cd717abdd847ab11fe88/276926,2022-07-27 13:04:24+00:00,0xc36442b4a4522e871399cd717abdd847ab11fe88,5.410000e-16,NaN,NaN,NaN,True
1,0x0000000000006e543164be036824fcf832e67e47,0x670d4dd2e6badfbbd372d0d37e06cd2852754a04/3841,NaN,NaN,NaN,2022-08-02 12:49:24+00:00,0.770,NaN,False
2,0x0000000000006e543164be036824fcf832e67e47,0x670d4dd2e6badfbbd372d0d37e06cd2852754a04/3980,NaN,NaN,NaN,2022-08-01 03:29:05+00:00,0.870,NaN,False
3,0x0000000000006e543164be036824fcf832e67e47,0x9231f13343f871b051fa5002e8c04beed91b4f62/1315,NaN,NaN,NaN,2022-09-20 17:56:35+00:00,0.347,NaN,False
4,0x0000000000006e543164be036824fcf832e67e47,0xc3bac22862ff04c7b5d6d9780958d0d754b0db7a/176...,2022-08-05 23:26:49+00:00,0xc3bac22862ff04c7b5d6d9780958d0d754b0db7a,6.000000e-02,NaN,NaN,NaN,True
...,...,...,...,...,...,...,...,...,...
14707380,0xffffffff7a5cb56bbccb9dfaff17bb2b26a3d71c,0xe51aac67b09eaed6d3d43e794d6bae679cbe09d8/4072,2022-07-22 15:49:34+00:00,0xe51aac67b09eaed6d3d43e794d6bae679cbe09d8,1.005000e+00,NaN,NaN,NaN,True
14707381,0xffffffff7a5cb56bbccb9dfaff17bb2b26a3d71c,0xe51aac67b09eaed6d3d43e794d6bae679cbe09d8/7121,2022-08-21 11:34:42+00:00,0xe51aac67b09eaed6d3d43e794d6bae679cbe09d8,5.400000e-01,NaN,NaN,NaN,True
14707382,0xffffffff7a5cb56bbccb9dfaff17bb2b26a3d71c,0xe51aac67b09eaed6d3d43e794d6bae679cbe09d8/7911,2022-07-29 15:54:44+00:00,0xe51aac67b09eaed6d3d43e794d6bae679cbe09d8,8.690000e-01,NaN,NaN,NaN,True
14707383,0xffffffff7a5cb56bbccb9dfaff17bb2b26a3d71c,0xe51aac67b09eaed6d3d43e794d6bae679cbe09d8/9948,NaN,NaN,NaN,2022-06-25 13:27:10+00:00,1.189,NaN,False


In [8]:
nft_flippers = pd.read_csv(f"{local_path}/nft_flippers.csv", header=None, names=['wallet_address'])

In [9]:
nft_flippers

,wallet_address
0,0x00003183f59e825911d98fb509a157cd2abbae25
1,0x0011a5b88073adf053ec6c602136e9b24b26b485
2,0x006236cc0510792b7f5bd3b0a5b0c50a764fbca9
3,0x007287045479ce07488b2ba6482af01754ae05ee
4,0x00c3ff532fee97d06b7ef8bf14db72fadb005007
...,...
2191,0xff274763062ab364fe17d17cc1cc10fa9e41e040
2192,0xff5ad07002d876bbb63a17d3f5bab8f63f034e1e
2193,0xff747758a76e56a641f7bc9dd87f810af773fef2
2194,0xffce05977d9bbb31a2afeb10980f7e35247d1121


In [29]:
%%time
flipper_nbps = non_batch_ps[non_batch_ps.wallet_address.isin(nft_flippers.wallet_address) 
                            & ~non_batch_ps.hold_time.isna()]

CPU times: user 524 ms, sys: 7.28 ms, total: 532 ms
Wall time: 530 ms


In [20]:
flipper_nbps

,wallet_address,nft_id,purchase_time,collection_id,purchase_value,sale_time,sale_value,hold_time,is_owned
6760,0x00003183f59e825911d98fb509a157cd2abbae25,0x5af0d9827e0c53e4799bb226655a1de152a425a5/1362,2022-04-27 05:26:27+00:00,0x5af0d9827e0c53e4799bb226655a1de152a425a5,0.6700,2022-05-14 00:03:27+00:00,0.657000,16 days 18:37:00,False
12651,0x0011a5b88073adf053ec6c602136e9b24b26b485,0x0c9153d9abb5e0489c89b38837f248816f151f42/1228,2022-04-05 07:15:53+00:00,0x0c9153d9abb5e0489c89b38837f248816f151f42,0.2500,2022-04-11 22:00:02+00:00,0.261000,6 days 14:44:09,False
12653,0x0011a5b88073adf053ec6c602136e9b24b26b485,0x0eda3c383f13c36db1c96bd9c56f715b09b9e350/5949,2022-06-04 04:11:59+00:00,0x0eda3c383f13c36db1c96bd9c56f715b09b9e350,0.1000,2022-06-04 04:39:04+00:00,0.080000,0 days 00:27:05,False
12654,0x0011a5b88073adf053ec6c602136e9b24b26b485,0x10eddda195fd33f81635d73df91679b642fbe7f0/313,2022-05-07 03:21:06+00:00,0x10eddda195fd33f81635d73df91679b642fbe7f0,0.0408,2022-05-07 03:40:46+00:00,0.068900,0 days 00:19:40,False
12657,0x0011a5b88073adf053ec6c602136e9b24b26b485,0x16356944741a719b02f63e8f001f7ed7ec79c969/533,2022-05-30 09:55:14+00:00,0x16356944741a719b02f63e8f001f7ed7ec79c969,0.1490,2022-05-30 09:59:34+00:00,0.184444,0 days 00:04:20,False
...,...,...,...,...,...,...,...,...,...
14700648,0xffdef5e9eeaceac0800a2f784105105dbaf1d884,0xfa242fff7490ada6cde6a8b3e686435982b4507d/5499,2022-07-10 02:30:43+00:00,0xfa242fff7490ada6cde6a8b3e686435982b4507d,0.0195,2022-07-10 13:50:33+00:00,0.015000,0 days 11:19:50,False
14700650,0xffdef5e9eeaceac0800a2f784105105dbaf1d884,0xfabe30e32ea8f9f7d297c6e3d6bdcf2d98c365e3/747,2022-08-18 16:33:24+00:00,0xfabe30e32ea8f9f7d297c6e3d6bdcf2d98c365e3,0.0065,2022-08-18 22:53:45+00:00,0.015000,0 days 06:20:21,False
14700651,0xffdef5e9eeaceac0800a2f784105105dbaf1d884,0xfb47683d2cca9ecc2d84674bb8658611fd56f277/1458,2022-08-29 23:50:16+00:00,0xfb47683d2cca9ecc2d84674bb8658611fd56f277,0.0070,2022-08-30 01:22:43+00:00,0.015000,0 days 01:32:27,False
14700653,0xffdef5e9eeaceac0800a2f784105105dbaf1d884,0xfb76f849baf4a4db22550ebc7beb520652f888a0/5646,2022-08-03 21:12:50+00:00,0xfb76f849baf4a4db22550ebc7beb520652f888a0,0.0060,2022-08-04 16:59:38+00:00,0.012000,0 days 19:46:48,False


In [34]:
profits = flipper_nbps['sale_value'] - flipper_nbps['purchase_value']

In [36]:
is_profit = profits > 0

In [46]:
flipper_profits = pd.concat([flipper_nbps['wallet_address'], profits, is_profit], axis=1)\
                        .rename(columns={0: 'profit', 1: 'is_profit'})

In [47]:
flipper_profits

,wallet_address,profit,is_profit
6760,0x00003183f59e825911d98fb509a157cd2abbae25,-0.013000,False
12651,0x0011a5b88073adf053ec6c602136e9b24b26b485,0.011000,True
12653,0x0011a5b88073adf053ec6c602136e9b24b26b485,-0.020000,False
12654,0x0011a5b88073adf053ec6c602136e9b24b26b485,0.028100,True
12657,0x0011a5b88073adf053ec6c602136e9b24b26b485,0.035444,True
...,...,...,...
14700648,0xffdef5e9eeaceac0800a2f784105105dbaf1d884,-0.004500,False
14700650,0xffdef5e9eeaceac0800a2f784105105dbaf1d884,0.008500,True
14700651,0xffdef5e9eeaceac0800a2f784105105dbaf1d884,0.008000,True
14700653,0xffdef5e9eeaceac0800a2f784105105dbaf1d884,0.006000,True


In [96]:
n_profit = flipper_profits.groupby('wallet_address').is_profit.sum().rename('n_profit')
avg_profit = flipper_profits.groupby('wallet_address').profit.mean().rename('avg_profit')
total_profit = flipper_profits.groupby('wallet_address').profit.sum().rename('tot_profit')

In [97]:
candidates = pd.concat([n_profit, avg_profit, total_profit], axis=1)

In [98]:
candidates

,n_profit,avg_profit,tot_profit
wallet_address,,,
0x00003183f59e825911d98fb509a157cd2abbae25,0,-0.013000,-0.013000
0x0011a5b88073adf053ec6c602136e9b24b26b485,42,0.014997,0.959786
0x006236cc0510792b7f5bd3b0a5b0c50a764fbca9,6,0.003939,0.027574
0x007287045479ce07488b2ba6482af01754ae05ee,6,0.075812,0.606500
0x00c3ff532fee97d06b7ef8bf14db72fadb005007,7,0.054278,0.488500
...,...,...,...
0xff274763062ab364fe17d17cc1cc10fa9e41e040,75,0.016478,1.664289
0xff5ad07002d876bbb63a17d3f5bab8f63f034e1e,220,0.024061,6.713146
0xff747758a76e56a641f7bc9dd87f810af773fef2,3,0.047033,0.141100


#### In order to recommend many NFTs, we need flippers with many profitable transactions

In [99]:
candidates.n_profit.describe()

count    2178.000000
mean       65.404500
std        88.179512
min         0.000000
25%        18.000000
50%        40.000000
75%        77.000000
max      1557.000000
Name: n_profit, dtype: float64

In [100]:
candidates = candidates[candidates.n_profit > candidates.n_profit.quantile(.75)]

In [101]:
candidates

,n_profit,avg_profit,tot_profit
wallet_address,,,
0x017347cb75ac8725608e593ea35d83f9b2b3cfb8,159,0.217827,47.921976
0x01b44b1018c0629ead48fc88c59d56f3894b1535,108,0.016378,3.013546
0x02736d5c8dcea65539993d143a3de90cebca9c3c,84,0.052650,9.740265
0x02efc390c8c60e978e991b15d71ebaa34783f090,103,0.115110,22.331291
0x0457bdd7d70c82f4cb1c850bd49064a544630978,156,-0.000610,-0.245372
...,...,...,...
0xfefef682c54d7dc0b7bede523db0a409729be2df,147,0.000841,0.353970
0xff0c68cdc0dd46a6ed8ba68e887a2a673c46f4e6,271,0.016293,8.521487
0xff2154f36a2bf303604fb2c77bd8548e4093d74f,127,0.021359,3.887249


#### High average means good recommendation for the users

In [102]:
candidates.avg_profit.describe()

count    544.000000
mean       0.154257
std        1.624673
min       -0.168967
25%        0.016273
50%        0.034505
75%        0.066675
max       28.751002
Name: avg_profit, dtype: float64

In [103]:
candidates = candidates[candidates.avg_profit > candidates.avg_profit.quantile(.50)]

In [104]:
candidates

,n_profit,avg_profit,tot_profit
wallet_address,,,
0x017347cb75ac8725608e593ea35d83f9b2b3cfb8,159,0.217827,47.921976
0x02736d5c8dcea65539993d143a3de90cebca9c3c,84,0.052650,9.740265
0x02efc390c8c60e978e991b15d71ebaa34783f090,103,0.115110,22.331291
0x04acfafe348c904f1791bd74db3fe4ec06f8a2f9,151,0.076821,13.981463
0x04b12f92aeb5f81c3412f9909c1784b1e7d3cdb2,100,0.080764,10.418600
...,...,...,...
0xfa89ec40699bbfd749c4eb6643dc2b22ff0e2aa6,175,1.222177,305.544168
0xfaf9f55e72d088f68cc01b2ff505abc11f6597b9,82,0.040684,6.550044
0xfc9b1720af55076e7e55a31d2e6a860dbdb54a3b,105,0.040820,6.327166


In [86]:
candidates.sort_values('tot_profit', ascending=False)

,n_profit,avg_profit,tot_profit
wallet_address,,,
0x13d8faf4a690f5ae52e2d2c52938d1167057b9af,806,28.751002,25099.624800
0xc310e760778ecbca4c65b6c559874757a4c4ece0,217,24.771922,5747.086000
0xfa89ec40699bbfd749c4eb6643dc2b22ff0e2aa6,175,1.222177,305.544168
0x2187f16f2d82da53581a2f48ebcffd415d68c997,381,0.220305,131.962728
0x5905096f0926dccdf68361f6ce40b21b3433d437,480,0.159496,103.034666
...,...,...,...
0x2fe2a75e518ffdc0a392f569440deb909f7010de,90,0.040098,4.731552
0x4760e5a9d5dca25593b2630e7093c5cfa7a640de,81,0.044220,4.598911
0x4687f6b19b5ceea0cd50eca6711127b23602deac,81,0.037356,4.295973


#### The top ones are making too much profit and are suspicious. Let's check them out

In [107]:
for address in candidates.sort_values('tot_profit', ascending=False).head(5).index:
    print(f'https://etherscan.io/address/{address}')

https://etherscan.io/address/0x13d8faf4a690f5ae52e2d2c52938d1167057b9af
https://etherscan.io/address/0xc310e760778ecbca4c65b6c559874757a4c4ece0
https://etherscan.io/address/0xfa89ec40699bbfd749c4eb6643dc2b22ff0e2aa6
https://etherscan.io/address/0x2187f16f2d82da53581a2f48ebcffd415d68c997
https://etherscan.io/address/0x5905096f0926dccdf68361f6ce40b21b3433d437


#### They are contracts. Let us fiter them out.

In [109]:
contracts = ['0x13d8faf4a690f5ae52e2d2c52938d1167057b9af', '0xc310e760778ecbca4c65b6c559874757a4c4ece0']
profitable_flippers = candidates[~candidates.index.isin(contracts)]

In [110]:
profitable_flippers.describe()

,n_profit,avg_profit,tot_profit
count,270.000000,270.000000,270.000000
mean,171.574074,0.100014,22.797977
std,110.282571,0.107200,25.665346
min,78.000000,0.034601,3.887910
25%,99.000000,0.046308,9.438099
50%,133.500000,0.065973,15.764811
75%,199.750000,0.108933,26.875230
max,781.000000,1.222177,305.544168


#### Even the least successful candidate made 8 ETH. We do need to filter further.

In [111]:
np.savetxt(f'{local_path}/profitable_nft_flippers.csv', profitable_flippers.index, delimiter=",", fmt='%s')

In [95]:
for address in candidates.sample(10).index:
    print(f'https://etherscan.io/address/{address}')

https://etherscan.io/address/0xb5f31eb9e70f88c5997a243dfb59b7097bcd84be
https://etherscan.io/address/0x2f7cbec4367d225dda665f501c269116f323e61d
https://etherscan.io/address/0x6c8c7539bf6a61c249c520c837ed0e19f91344dc
https://etherscan.io/address/0x378f7ade12d33a12aa0bce82ee1131812723c37d
https://etherscan.io/address/0xba7933402348a902064499ed883c49843eeb7019
https://etherscan.io/address/0x4fb5ae963c4fda03d98fc3ff8ff287fdf49de548
https://etherscan.io/address/0x170f20ea829d997f144a85ea43904de8050a8f73
https://etherscan.io/address/0x5905096f0926dccdf68361f6ce40b21b3433d437
https://etherscan.io/address/0x314efdc75b58e1e09c187ac3cee9e75a42d4590d
https://etherscan.io/address/0x24f65fa44edd3165c221b370c055111c9aede2de
